# 영양성분 데이터셋 수집 및 전처리

In [291]:
import pandas as pd
pd.set_option('display.max_colwidth', 700)
import numpy as np
import re

import requests
import json
# from pandas.io.json import json_normalize
from pandas import json_normalize

# 경고 메시지를 무시
import warnings
warnings.filterwarnings(action='ignore')

import pickle

In [292]:
# Open API 데이터셋 호출 용 Key
# !pip install python-dotenv # 설치시 사용
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

OPENAPI_KEY = os.environ.get('OPENAPI_KEY')

## 1. Open API에서 데이터를 가져와서 DataFrame 생성

In [293]:
# OPEN API에서 데이터 가져오기
# 건강기능식품 품목분류정보
df = pd.DataFrame()
keyId = OPENAPI_KEY # 개인 key 적용
serviceId = 'I2710'
dataType = 'json'
startIdx = 1
endIdx = 1000

while True:
    try:
        url = f'https://openapi.foodsafetykorea.go.kr/api/{keyId}/{serviceId}/{dataType}/{startIdx}/{endIdx}'

        response = requests.get(url)
        contents = response.text

        # 출력된 문자열 json 파일로 변환
        json_ob = json.loads(contents)

        # 필요한 내용은 serviceId > 'row' > key에 들어있음
        body = json_ob[serviceId]['row']

        # 추출된 값 DataFrame으로 변환
        df2 = json_normalize(body)

        # 기존 DF에 추가
        df = pd.concat([df, df2], ignore_index=True)

        # 해당하는 데이터가 없을 경우 루프 종료
        if endIdx > int(json_ob[serviceId]['total_count']):
            break

        startIdx += 1000
        endIdx += 1000

    except:
        url = f'https://openapi.foodsafetykorea.go.kr/api/{keyId}/{serviceId}/{dataType}/{startIdx}/{endIdx}'

        response = requests.get(url)
        contents = response.text

        # 출력된 문자열 json 파일로 변환
        json_ob = json.loads(contents)

        # 필요한 내용은 serviceId > 'row' > key에 들어있음
        body = json_ob[serviceId]['row']

        # 추출된 값 DataFrame으로 변환
        df2 = json_normalize(body)

        # 기존 DF에 추가
        df = pd.concat([df, df2], ignore_index=True)

        # 해당하는 데이터가 없을 경우 루프 종료
        if endIdx > int(json_ob[serviceId]['total_count']):
            break

        startIdx += 1000
        endIdx += 1000

In [294]:
df.columns = ['주된기능성', '성분명', '최종수정일', '최초등록일',
              '품목명', '섭취시주의사항', '비고', '일일섭취량하한',
              '일일섭취량상한', '섭취단위']

df = df[['품목명', '성분명', '최종수정일', '최초등록일', 
         '주된기능성', '섭취시주의사항', '비고', '일일섭취량상한', '일일섭취량하한', '섭취단위']]

df.head()

,품목명,성분명,최종수정일,최초등록일,주된기능성,섭취시주의사항,비고,일일섭취량상한,일일섭취량하한,섭취단위
0,아라비아검(아카시아검),아라비아검(아카시아검)식이섬유,20200820,20080922,배변활동 원활에 도움을 줄 수 있음,반드시 충분한 물과 함께 섭취할 것 (액상제외),,20.01,19.99,g
1,베타카로틴,,20210924,20080626,(1) 어두운 곳에서 시각 적응을 위해 필요\n(2) 피부와 점막을 형성하고 기능을 유지하는데 필요\n(3) 상피세포의 성장과 발달에 필요,(1) 흡연자는 섭취 시 전문가와 상담할 것\n(2) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것,식품원료를 사용하여 제조가공한 경우는 1.26mg 이상(상한섭취량 없음),7,0.42,mg
2,비오틴,,20210924,20080626,"(1) 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요",(1) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것,,0.9,0.009,mg
3,공액리놀레산,공액리놀레산,20200820,20100112,①과체중인 성인의 체지방 감소에 도움을 줄 수 있음,"위장장애가 발생할 수 있음\n영·유아, 임산부는 섭취를 삼가야 함\n식사조절, 운동을 병행하는 것이 체지방 감소에 효과적임",,4.2,1.4,g
4,가르시니아캄보지아 추출물,총(-)-Hydroxycitric acid,20200820,20100224,탄수화물이 지방으로 합성되는 것을 억제하여 체지방 감소에 도움을 줄 수 있음,"(가) 어린이, 임산부 및 수유부는 섭취를 피할 것\n(나) 간·신장·심장질환, 알레르기 및 천식이 있거나 의약품 복용 시 전문가와 상담할 것\n(다) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것",,2800,750,mg


## 2. 영양성분 데이터셋 분석
- 총 434행
- 전처리가 필요한 내용 확인

### 2-1. '품목명' 컬럼

In [295]:
# '품목명' 컬럼에서 중복값 발견
df[df.duplicated('품목명', keep=False)] # 사삼추출물(제2022-7호)의 경우 2개의 값 발견

,품목명,성분명,최종수정일,최초등록일,주된기능성,섭취시주의사항,비고,일일섭취량상한,일일섭취량하한,섭취단위
175,사삼추출물(제2022-7호),"리놀레산(Linoleic acid), 베타시토스테롤(β-Sitosterol)",20220221,20220221,(국문) 체지방 감소에 도움을 줄 수 있음 (영문) May help to reduce body fat,"(1) 영‧유아, 어린이, 임산부 및 수유부는 섭취에 주의할 것 (2) 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의 (3) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것 (4) 식사조절, 운동을 병행하는 것이 체지방 감소에 효과적임",제2022-7호,750,750,mg
373,사삼추출물(제2022-7호),"리놀레산(Linoleic acid), 베타시토스테롤(β-Sitosterol)",20221125,20221125,(국문) 체지방 감소에 도움을 줄 수 있음\n(영문) May help to reduce body fat,"(1) 영‧유아, 어린이, 임산부 및 수유부는 섭취에 주의할 것\n(2) 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의\n(3) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n(4) 식사조절, 운동을 병행하는 것이 체지방 감소에 효과적임",기능성 원료 인정(제2022-7호),750,750,mg/일


- 최초등록일과 최종수정일이 최신인 데이터 외 중복 행 삭제 필요

In [296]:
# '품목명' 컬럼에서 결측치 확인 -> 제거 필요 / "" 공백문자가 들어가있어서 결측치로 안잡힘
df[df['품목명'] == ""]

,품목명,성분명,최종수정일,최초등록일,주된기능성,섭취시주의사항,비고,일일섭취량상한,일일섭취량하한,섭취단위
171,,,20211009,20211009,,,,1000,100,g


- 품목명이 결측치인 경우 해당 행 제거 필요

### 2-2. '주된기능성' 컬럼

In [297]:
# '품목명'은 있고, '주된기능성' 컬럼값이 없는 경우 발견 => 값 하드코딩
df[(df['품목명'] != "") & (df['주된기능성'] == "")] 

,품목명,성분명,최종수정일,최초등록일,주된기능성,섭취시주의사항,비고,일일섭취량상한,일일섭취량하한,섭취단위
162,차조기등복합추출물(New제2007-13호),,20201029,20201029,,,,2400,2400,mg


- 해당 값은 Open API 자료 제공 사이트에서 값을 확인 후 하드코딩하기로 결정

### 2-3. '섭취단위' 컬럼

In [298]:
# '섭취단위' 컬럼에서 숫자 발견 -> '1 g/일' : 단위가 아닌 숫자는 전처리 과정에서 제거 필요
df['섭취단위'].value_counts()

mg         173
g          104
mg/일        99
g/일         34
CFU         10
CFU/일        9
ml           5
μg           2
mg/g         2
mg α-TE      1
μg RAE       1
mg/day       1
             1
IU           1
1 g/일        1
Name: 섭취단위, dtype: int64

- 섭취단위가 아닌 숫자는 전처리 과정에서 제거 필요
  - e.g : '1 g/일' -> 'g/일'

In [299]:
# 섭취단위가 결측치인 경우
df[df['섭취단위'] == ""]

,품목명,성분명,최종수정일,최초등록일,주된기능성,섭취시주의사항,비고,일일섭취량상한,일일섭취량하한,섭취단위
165,프로바이오틱스(드시모네)(제2009-28호),"Streptococcus thermophillus DSM24731, Lactobacillus acidophillus DSM24735, Lactobacillus delbrueckii ssp bulgaricus DSM24734, Lactobacillus paracasei DSM24733, Lactobacillus plantarum DSM24730, Bifidobacterium longum DSM24736, Bifidobacterium infantis DSM24737, Bifidobacterium breve DSM24732",20201029,20201029,"- 유익한 유산균 증식, 유해균 억제, 배변활동 원활\n- 장 면역을 조절하여 장 건강에 도움을 줄 수 있음",,,3000000000000,100000000,


- 섭취단위가 결측치이고, 주된기능성에 '유산균'이 포함된 경우 유산균 단위인 'CFU'로 지정 필요

### 2-4. '비고' 컬럼

In [300]:
# '비고' 컬럼 unique 값 확인
df['비고'].unique()

array(['', '식품원료를 사용하여 제조가공한 경우는 1.26mg 이상(상한섭취량 없음)',
       '최종제품 중 비타민 A 및 D의 함량이 제 3. 1. 1-1 비타민 A 및 1-3 비타민 D 일일섭취량의 최대함량을 넘지 않도록 하여야 함(비타민 A 및 D의 함량이 반드시 표시되어야 함)',
       '2021.7.11: 기능성 내용 변경', '니코틴산 4.5～23 mg, 니코틴산아미드 4.5～670 mg',
       '21.7.11: 기능성 내용 변경',
       '혈중 콜레스테롤 개선에 도움을 줄 수 있음 : 차전자피 식이섬유로서 6.0 g 이상, 배변활동 원활에 도움을 줄 수 있음 : 차전자피 식이섬유로서 5.0 g 이상',
       '항산화에 도움을 줄 수 있음 : 총 엽록소로서 10~80 mg, 혈중 콜레스테롤 개선에 도움을 줄 수 있음 : 총 엽록소로서 40~80 mg',
       '피부건강·항산화에 도움을 줄 수 있음 : 총 엽록소로서 8〜150 mg, 면역력 증진·혈중 콜레스테롤 개선에 도움을 줄 수 있음 : 총 엽록소로서 125〜150 mg',
       '보이차추출물(제2014-7호)',
       '혈중 콜레스테롤 개선·식후 혈당상승 억제에 도움을 줄 수 있음 : 이눌린/치커리추출물 식이섬유로서 7.2～20 g, 배변활동 원활에 도움을 줄 수 있음 : 이눌린/치커리추출물식이섬유로서 6.4～20 g',
       '일일섭취량 : 3～10 μg(120∼400 IU)',
       '혈중 콜레스테롤 개선·식후 혈당상승 억제·배변활동 원활에 도움을 줄 수 있음 : 구아검/구아검가수분해물 식이섬유로서 9.9～27 g,  장내 유익균 증식에 도움을 줄 수 있음 : 구아검/구아검가수분해물 식이섬유로서 4.6～27 g',
       '혈중 콜레스테롤 개선에 도움을 줄 수 있음 : 키토산 또는 키토올리고당으로서 1.2～4.5 g, 체지방 감소에 도움을 줄 수 있음 : 키토산으로서 3.0∼4.5 g, 키토올리고당으로서 

- 섭취시 주의사항, 주된기능성, 혹은 개별인정원료인 경우에는 인정번호에 대한 내용이 포함되어 있음

In [301]:
df[df['비고'].str.contains('취소')][['품목명', '비고']]

,품목명,비고
299,녹차추출물/테아닌 복합물(제2010-51호),건강기능식품 기능성 원료 인정서 자진반납에 따른 인정사항 취소


- '인정사항'과 함께 '취소', 혹은 '폐지'라는 문자열이 함께 존재할 경우 해당 영양성분은 인정취소 또는 인정 폐지된 것으로 보고 해당 행 제거

## 3. 영양성분 데이터셋 통합 전처리

### 3-1. '주된기능성' 컬럼 하드코딩
- 결측치의 경우 건강기능식품 정보포털 내용을 바탕으로 하드코딩

In [302]:
# https://www.hffinfo.com/search/raw_material/2007-13?page=1&limit=10&query=%EC%B0%A8%EC%A1%B0%EA%B8%B0

mask = df['품목명'] == '차조기등복합추출물(New제2007-13호)'
df.loc[mask, ['주된기능성']] = ['관절 건강에 도움을 줄 수 있음']
df.loc[mask]

,품목명,성분명,최종수정일,최초등록일,주된기능성,섭취시주의사항,비고,일일섭취량상한,일일섭취량하한,섭취단위
162,차조기등복합추출물(New제2007-13호),,20201029,20201029,관절 건강에 도움을 줄 수 있음,,,2400,2400,mg


In [303]:
# 인정폐지된 영양성분 명 - 업데이트 할 리스트
del_ingredient_names = ['녹차추출물/테아닌 복합물(제2010-51호)', '배초향 추출물(Agatri)(제2020-5호)', 
             '수국잎열수추출물(제2020-8호)', 'L-글루타민(제2008-7호)']

### 3-2. 영양성분 데이터셋 기본 전처리
- 2에서 분석한 내용을 바탕으로 기본 전처리
- DB에 들어갈 영문 컬럼명으로 수정

In [304]:
# 데이터프레임 전처리 함수 정의
def processing_df(df):
    # 최초등록일, 최종수정일 기준으로 오름차순 후 인덱스 재정렬 -> 최신일 수록 인덱스가 더 높은 숫자
    df = df.sort_values(by=['최초등록일'], ascending=True, ignore_index=True)
    df = df.sort_values(by=['최종수정일'], ascending=True, ignore_index=True)
    
    # "" 공백문자로 되어있는 것이 결측치를 의미. ""를 NaN로 대체
    df.replace("", np.nan, inplace=True)
    
    # '품목명' 컬럼에 결측치가 있을 경우 해당 행 제거 및 인덱스 재정렬
    if df['품목명'].isnull().sum() > 0:
        df.dropna(subset=['품목명'], axis=0, how='any', inplace=True)
        
    # '품목명' 컬럼에 중복값이 있을 경우 
    if df['품목명'].duplicated(keep=False).sum() > 1:
        # '최초등록일'과 '최종수정일' 컬럼의 값이 더 최신인 행을 남겨두고 나머지 중복행 제거 
        # '최초등록일'과 '최종수정일' 컬럼을 기준으로 오름차순으로 정렬된 데이터프레임 중 마지막을 제외한 중복행 제거
        df.drop_duplicates(subset=['품목명'], keep='last', inplace=True, ignore_index=True)
    
    
    # '섭취단위' 컬럼이 결측치이고, 주된기능성 컬럼에 '유산균' 문자열이 있는 경우 -> 'CFU'로 결측치 대체
    mask_1 = (df['섭취단위'].isnull()) & (df['주된기능성'].str.contains('유산균'))
    if mask_1.sum() >0:
        df.drop(df[mask_1].index, axis=0, inplace=True)
    
    # '섭취단위' 컬럼에 단위와 관련된 문자열 외 숫자가 있는 경우 제거
    df['섭취단위'] = df['섭취단위'].apply(lambda x: re.sub(r'\d', '', x))
    
    # '비고' 컬럼에 '인정사항'이라는 문자열과 '취소' 또는 '폐지'라는 문자열이 같이 나오는 경우 -> 해당 행 제거
    mask_2 = (df['비고'].str.contains('인정사항')) & ((df['비고'].str.contains('취소')) | (df['비고'].str.contains('폐지')))
    if (mask_2).sum() > 0:
        df.drop(df[mask_2].index, axis=0, inplace=True)
        
    # 인정취소/폐지된 영양성분 명이 있는 행 제거
    mask_3 = df['품목명'].isin(del_ingredient_names)

    if mask_3.any():
        df.drop(df[mask_3].index, axis=0, inplace=True)
        
    # 인덱스 재정렬
    df.reset_index(drop=True, inplace=True)
    
    # 필요한 컬럼만 가져오기
    df = df[['품목명', '주된기능성', '섭취시주의사항', '일일섭취량상한', '일일섭취량하한', '섭취단위', '비고']]
    
    # 컬럼명 영문으로 다시 수정
    df.columns = ['ingredient_raw_name', 'ingredient_function', 'ingredient_caution', 'ingredient_limit_high', 'ingredient_limit_low', 'ingredeint_unit', 'ingredient_etc']
    
    return df

In [305]:
i_df = processing_df(df)
df.shape, i_df.shape # 442행 -> 435행

((444, 10), (437, 7))

## 4. 영양성분 명 전처리
- `ingredient_raw_name` : OPEN API RAW 데이터
- `ingredient_name` : 영양제 상세페이지 내 영양성분 명으로 들어갈 데이터
- `ingredient_grp_name` : 영양성분 추천 리포트 관련 프론트에서 영양성분 명으로 들어갈 데이터
- `ingredeint_auth_num` : 영양성분 중 개별인정원료의 인정번호
- `ingredient_recom_name` : 추천 시스템에서 사용될 영양성분 명(비슷한 성분 통합 및 특수문자 등 제거)

### 4-1. 개별인정원료의 경우 인정번호 추출
- 추출한 인정번호 `ingredient_auth_num` 컬럼에 추가
- 인정번호가 삭제된 name `'ingredient_name_re'` 저장

In [306]:
# 영양성분명에서 개별인정원료의 인정번호 추출해서 개별 컬럼에 저장하고, 인정번호가 제거된 영양성분명 개별 컬럼에 저장된 df 구하는 함수 정의
def extract_auth_num(df, raw_name_col, cleaned_name_col, auth_num_col):
    # 기능성 원료 인정번호 패턴
    auth_num_pattern = r'\([^()]*제[0-9\-]+호[^()]*\)'

    cleand_auth_nums = []
    cleaned_names = []

    for raw_name in df[raw_name_col]:
    # 1. 'ingredient_raw_name'컬럼 텍스트에서 개별인정원료의 인정번호 추출
        auth_num_match = re.search(auth_num_pattern, raw_name)
        if auth_num_match:
            auth_num = re.findall(r'제[0-9\-]+호', auth_num_match.group()) # group()은 re.search()의 결과인 object에서 실제로 매칭된 문자열을 반환
            auth_num_str = ''.join(auth_num) # findall은 리스트로 반환하기 때문에 문자열로 변환
            cleaned_auth_num_str = re.sub(r'\s', '', auth_num_str) # whitespace 삭제
            cleand_auth_nums.append(cleaned_auth_num_str) # cleand_auth_nums 리스트에 추가
        else:
            cleand_auth_nums.append(np.nan) # 개별인정원료 인정번호가 없을 경우 결측치로 대체
        
        # 2. 'ingredient_raw_name'컬럼 텍스트에서 기능성 원료 인정번호 삭제
        cleaned_name = re.sub(auth_num_pattern, '', raw_name)
        # cleaned_names 리스트에 추가
        cleaned_names.append(cleaned_name) 

    # 기능성 원료 인정번호만 추출한 문자열이 담긴 리스트를 컬럼값으로 추가
    df[auth_num_col] = cleand_auth_nums

    # 기능성 원료 인정번호 삭제된 문자열이 담긴 리스트를 컬럼값으로 추가 => 서비스상에 노출될 텍스트
    df[cleaned_name_col] = cleaned_names
    return df

In [307]:
# 인정번호 추출하는 함수 적용
i_df = extract_auth_num(i_df, 'ingredient_raw_name', 'ingredient_name', 'ingredient_auth_num')

# 인정번호 잘 추출 되었는지 확인
# check_columns = ['ingredient_raw_name', 'ingredient_name',  'ingredient_auth_num', 'ingredient_type']
# i_df.loc[(i_df['ingredient_type']==1)&(i_df['ingredient_raw_name'].str.contains('\(제')), check_columns]
check_columns = ['ingredient_raw_name', 'ingredient_name',  'ingredient_auth_num']
i_df.loc[(i_df['ingredient_raw_name'].str.contains('\(제')), check_columns]

,ingredient_raw_name,ingredient_name,ingredient_auth_num
14,커피만노올리고당분말(제2009-15호),커피만노올리고당분말,제2009-15호
15,식물스타놀에스테르(제2006-11호),식물스타놀에스테르,제2006-11호
16,석류추출물(제2010-23호),석류추출물,제2010-23호
17,L-글루타민산 유래 GABA 함유 분말(제2009-1호),L-글루타민산 유래 GABA 함유 분말,제2009-1호
18,중쇄지방산(MCFA) 함유 유지(제2009-20호),중쇄지방산(MCFA) 함유 유지,제2009-20호
...,...,...,...
432,백편두추출분말(NOVAponinⓇ)(제2024-6호),백편두추출분말(NOVAponinⓇ),제2024-6호
433,발효굴추출물(제2023-41호),발효굴추출물,제2023-41호
434,풋사과 농축분말(제2024-4호),풋사과 농축분말,제2024-4호
435,기장밀추출복합물(KeranatTM)(제2024-16호),기장밀추출복합물(KeranatTM),제2024-16호


In [308]:
# 'New'가 들어간 경우 인정번호 잘 추출 되었는지 확인
i_df.loc[(i_df['ingredient_raw_name'].str.contains('New')), check_columns]

,ingredient_raw_name,ingredient_name,ingredient_auth_num
123,차조기등복합추출물(New제2007-13호),차조기등복합추출물,제2007-13호
144,PMO 참밀알부민(New제2007-23호),PMO 참밀알부민,제2007-23호


In [309]:
# 인정번호 중 중복된 값이 있는지 확인 => 제거 필요
drop_maks = (i_df['ingredient_auth_num'].notnull()) & (i_df.duplicated('ingredient_auth_num', keep='last'))
# if drop_maks.any():
    
i_df[~drop_maks]

,ingredient_raw_name,ingredient_function,ingredient_caution,ingredient_limit_high,ingredient_limit_low,ingredeint_unit,ingredient_etc,ingredient_auth_num,ingredient_name
0,아연,①정상적인 면역기능에 필요②정상적인 세포분열에 필요,NaN,12,2.55,mg,NaN,NaN,아연
1,철,①체내 산소운반과 혈액생성에 필요②에너지 생성에 필요,특히 6세 이하는 과량섭취하지 않도록 주의,15,3.6,mg,NaN,NaN,철
2,비타민 B6,①단백질 및 아미노산 이용에 필요②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요,NaN,67,0.45,mg,NaN,NaN,비타민 B6
3,마그네슘,①에너지 이용에 필요②신경과 근육 기능 유지에 필요,NaN,250,94.5,mg,NaN,NaN,마그네슘
4,요오드,①갑상선 호르몬의 합성에 필요②에너지 생성에 필요③신경발달에 필요,NaN,0.15,0.045,mg,NaN,NaN,요오드
...,...,...,...,...,...,...,...,...,...
432,백편두추출분말(NOVAponinⓇ)(제2024-6호),(국문) 위 점막을 보호하여 위 건강에 도움을 줄 수 있음\n(영문) May help to maintain healthy stomach by protecting gastric mucosa,(1) 임산부 및 수유부는 섭취를 피할 것\n(2) 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의할 것\n(3) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것,715,715,mg/일,제2024-6호,제2024-6호,백편두추출분말(NOVAponinⓇ)
433,발효굴추출물(제2023-41호),(국문) 근력개선에 도움을 줄 수 있음\n(영문) May help to improve muscle strength,"(1) 영·유아, 어린이, 임산부 및 수유부는 섭취를 피할 것\n(2) 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의\n(3) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것 \n(4) 질환이 있거나 의약품 복용 시 섭취 전 전문가와 상담할 것",1000,1000,mg/일,제2023-41호,제2023-41호,발효굴추출물
434,풋사과 농축분말(제2024-4호),"(국문) ① 피부 보습에 도움을 줄 수 있음, ② 자외선에 의한 피부손상으로부터 피부건강 유지에 도움을 줄 수 있음\n(영문) ① May help to maintain skin moisturizing, ② May help to maintain skin health from skin damage by UV radiation","(1) 영·유아, 어린이, 임산부 및 수유부는 섭취를 피할 것 \n(2) 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의할 것\n(3) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것",1,1,g/일,제2024-4호,제2024-4호,풋사과 농축분말
435,기장밀추출복합물(KeranatTM)(제2024-16호),(국문) 모발상태(윤기·탄력) 개선에 도움을 줄 수 있음\n(영문) May help to improve hair condition (luster·elasticity),"(1) 영·유아, 어린이, 임산부 및 수유부는 섭취를 피할 것\n(2) 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의\n(3) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것",300,300,mg/일,제2024-16호,제2024-16호,기장밀추출복합물(KeranatTM)


- Open API 원본 데이터에서 최초등록일이 더 최신인 데이터를 제외하고 중복된 행 제거(현재 오름차순 정렬됨)
- 중복데이터 없애기 위해 함수 수정

In [310]:
# 영양성분명에서 개별인정원료의 인정번호 추출해서 개별 컬럼에 저장하고, 인정번호가 제거된 영양성분명 개별 컬럼에 저장된 df 구하는 함수 정의
def extract_auth_num(df, raw_name_col, cleaned_name_col, auth_num_col):
    # 기능성 원료 인정번호 패턴
    auth_num_pattern = r'\([^()]*제[0-9\-]+호[^()]*\)'

    cleand_auth_nums = []
    cleaned_names = []

    for raw_name in df[raw_name_col]:
    # 1. 'ingredient_raw_name'컬럼 텍스트에서 개별인정원료의 인정번호 추출
        auth_num_match = re.search(auth_num_pattern, raw_name)
        if auth_num_match:
            auth_num = re.findall(r'제[0-9\-]+호', auth_num_match.group()) # group()은 re.search()의 결과인 object에서 실제로 매칭된 문자열을 반환
            auth_num_str = ''.join(auth_num) # findall은 리스트로 반환하기 때문에 문자열로 변환
            cleaned_auth_num_str = re.sub(r'\s', '', auth_num_str) # whitespace 삭제
            cleand_auth_nums.append(cleaned_auth_num_str) # cleand_auth_nums 리스트에 추가
        else:
            cleand_auth_nums.append(np.nan) # 개별인정원료 인정번호가 없을 경우 결측치로 대체
        
        # 2. 'ingredient_raw_name'컬럼 텍스트에서 기능성 원료 인정번호 삭제
        cleaned_name = re.sub(auth_num_pattern, '', raw_name)
        # cleaned_names 리스트에 추가
        cleaned_names.append(cleaned_name) 

    # 기능성 원료 인정번호만 추출한 문자열이 담긴 리스트를 컬럼값으로 추가
    df[auth_num_col] = cleand_auth_nums

    # 기능성 원료 인정번호 삭제된 문자열이 담긴 리스트를 컬럼값으로 추가 => 서비스상에 노출될 텍스트
    df[cleaned_name_col] = cleaned_names
    
    # 인정번호가 결측치가 아닐 때 중복될 경우 '최초등록일'과 '최종수정일' 컬럼을 기준으로 오름차순으로 정렬된 데이터프레임 중 마지막을 제외한 중복행 제거(최신인 행을 남김)
    drop_mask = (df[auth_num_col].notnull()) & (df.duplicated(auth_num_col, keep='last')) # notnull() 이 없을 경우 결측치도 중복값으로 보고 제거될 수 있음
    if drop_mask.any():
        df = df[~drop_mask] # 제거할 행을 제외한 데이터프레임만 남긴다.
        df.reset_index(drop=True, inplace=True)
        
    return df
    

In [311]:
# 인정번호 추출하는 함수 적용
i_df = extract_auth_num(i_df, 'ingredient_raw_name', 'ingredient_name', 'ingredient_auth_num')

# 인정번호 중 중복된 값이 있는지 다시 확인 => 제거된 것을 확인
i_df[(i_df['ingredient_auth_num'].notnull()) & (i_df.duplicated('ingredient_auth_num', keep=False))] # 제거된 것을 확인
i_df.loc[i_df['ingredient_auth_num'] == '제2019-19호'] # 1개만 존재

,ingredient_raw_name,ingredient_function,ingredient_caution,ingredient_limit_high,ingredient_limit_low,ingredeint_unit,ingredient_etc,ingredient_auth_num,ingredient_name
351,댕댕이나무열매(허니베리)추출분말(제2019-19호),(국문) ① 비알콜성 간 손상으로부터 간 건강에 도움을 줄 수 있음\n ② 체지방 감소에 도움을 줄 수 있음\n(영문) ① May help to liver health by non-alcoholic liver injury\n ② May help to reduce body fat,"(1) 영·유아, 어린이, 임산부 및 수유부는 섭취에 주의 \n(2) 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의\n(3) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것",1960,1960,mg/일,NaN,제2019-19호,댕댕이나무열매(허니베리)추출분말


In [312]:
i_df.shape # 435행 => 인정번호 중복행 제거로 434개

(436, 9)

### 4-2. `ingredient_name` 컬럼 값에 특수문자 확인
- 프론트에서 보여줄 때 특수문자 확인

In [313]:
# 데이터프레임의 특정 컬럼값에 존재하는 특수문자를 리스트 형태로 가져오는 함수정의
def find_special_chars(df, col_name):
    # notnull()로 결측치가 아닌 문자열만 확인해야 오류가 나지 않는다.
    df = df.loc[df[col_name].notnull(), col_name]
    
    # 해당 컬럼의 모든 문자열 결합
    combined_text = ' '.join(df)
    
    # 특수문자만 추출
    special_chars = re.findall(r'[^가-힣A-Za-z0-9\s]', combined_text)
    
    # 추출된 특수문자를 집합(set)으로 변환하여 중복 제거 후 다시 list로 가져온다.
    unique_special_chars = list(set(special_chars))
    
    return unique_special_chars

In [314]:
print(find_special_chars(i_df, 'ingredient_name'))

['[', '.', 'Ⓡ', '>', ']', '-', '․', '·', '<', '/', '(', ')', '®', 'β', '%', ',']


In [315]:
# 'ingredient_grp_name'에 포함된 특수문자가 있는 행 확인
i_df.loc[i_df['ingredient_name'].str.contains(r'[®Ⓡ]'), check_columns] # '®'=> 'Ⓡ'로 대체

,ingredient_raw_name,ingredient_name,ingredient_auth_num
173,Green Mate Extract EFLA® 920(제2008-52호),Green Mate Extract EFLA® 920,제2008-52호
187,호로파종자추출물(TestofenⓇ)(제2019-15호),호로파종자추출물(TestofenⓇ),제2019-15호
240,밀 추출물(Ceratiq®)(제2020-15호),밀 추출물(Ceratiq®),제2020-15호
265,돌외잎주정추출분말(액티포닌®)(제2013-8호),돌외잎주정추출분말(액티포닌®),제2013-8호
308,모로오렌지추출분말(Morosil®)(제2021-7호),모로오렌지추출분말(Morosil®),제2021-7호
324,리스펙타(Respecta®)[프로바이오틱스 등 복합물](제2019-26호),리스펙타(Respecta®)[프로바이오틱스 등 복합물],제2019-26호
335,천심련추출물(ParActinⓇ)(제2022-3호),천심련추출물(ParActinⓇ),제2022-3호
336,타히보 추출물 (TabetriⓇ)(제2022-2호),타히보 추출물 (TabetriⓇ),제2022-2호
337,유산균복합물(AB-LIFE®)(제2022-15호),유산균복합물(AB-LIFE®),제2022-15호
346,가자추출물(AyuFlexⓇ)(제2022-12호),가자추출물(AyuFlexⓇ),제2022-12호


In [316]:
i_df.loc[i_df['ingredient_name'].str.contains(r'[<[]'), check_columns] # 꺽쇠 기호('<', '>')는 소괄호로 대체, 대괄호는 그대로 사용

,ingredient_raw_name,ingredient_name,ingredient_auth_num
213,피크노제놀-프랑스해안송껍질추출물<PYCNOGENOL>(제2006-1호),피크노제놀-프랑스해안송껍질추출물<PYCNOGENOL>,제2006-1호
324,리스펙타(Respecta®)[프로바이오틱스 등 복합물](제2019-26호),리스펙타(Respecta®)[프로바이오틱스 등 복합물],제2019-26호


In [317]:
i_df.loc[i_df['ingredient_name'].str.contains(r'\.'), check_columns] 

,ingredient_raw_name,ingredient_name,ingredient_auth_num
41,뮤코다당.단백,뮤코다당.단백,NaN
105,보스웰리아 추출물(Boswellia serrata R. extract)(제2014-23호),보스웰리아 추출물(Boswellia serrata R. extract),제2014-23호
221,과채유래유산균(L.plantarum CJLP133)(제2013-11호),과채유래유산균(L.plantarum CJLP133),제2013-11호
258,L.plantarum IM76과 B.longum IM55 복합물(NVP1703)(제2019-28호),L.plantarum IM76과 B.longum IM55 복합물(NVP1703),제2019-28호
354,B.breve IDCC 4401(BBR4401) 열처리배양건조물(제2021-5호),B.breve IDCC 4401(BBR4401) 열처리배양건조물,제2021-5호
387,L. plantarum LC27과 B. longum LC67의 프로바이오틱스 복합물(NVP-1702)(제2023-19호),L. plantarum LC27과 B. longum LC67의 프로바이오틱스 복합물(NVP-1702),제2023-19호
395,L. curvatus HY7601와 L. plantarum KY1032의 프로바이오틱스 복합물(제2019-4호),L. curvatus HY7601와 L. plantarum KY1032의 프로바이오틱스 복합물,제2019-4호


- '.' 는 '.' 기호로 통일하여 대체(영양제 데이터셋에서도 대부분 '.' 기호로 사용)
- '뮤코다당.단백'의 경우만 '·'로 대체

In [318]:
i_df.loc[i_df['ingredient_name'].str.contains(r'\%'), check_columns] # 그대로 사용 => 루테인 관련 제품은 따로 하드코딩 진행 예정

,ingredient_raw_name,ingredient_name,ingredient_auth_num
234,루테인지아잔틴복합추출물20%(제2013-23호),루테인지아잔틴복합추출물20%,제2013-23호


In [319]:
i_df.loc[i_df['ingredient_name'].str.contains(r'\․'), check_columns] # '․' 는 구분자로 '·'로 대체

,ingredient_raw_name,ingredient_name,ingredient_auth_num
172,콩․보리 발효복합물(제2015-10호),콩․보리 발효복합물,제2015-10호


In [320]:
i_df.loc[i_df['ingredient_name'].str.contains(r'\-'), check_columns] # '-' 는 구분자로 그대로 사용

,ingredient_raw_name,ingredient_name,ingredient_auth_num
17,L-글루타민산 유래 GABA 함유 분말(제2009-1호),L-글루타민산 유래 GABA 함유 분말,제2009-1호
38,"NAG(엔에이지, N-아세틸글루코사민, N-Acetylglucosamine)","NAG(엔에이지, N-아세틸글루코사민, N-Acetylglucosamine)",NaN
113,크랜베리 추출물(Cran-Max)(제2010-39호),크랜베리 추출물(Cran-Max),제2010-39호
116,보리 베타글루칸 추출물(Barley b-glucan Extract)(제2010-32호),보리 베타글루칸 추출물(Barley b-glucan Extract),제2010-32호
119,MR-10 민들레등복합추출물(제2013-31호),MR-10 민들레등복합추출물,제2013-31호
125,쇠비름주정추출분말(KDC16-2)(제2019-18호),쇠비름주정추출분말(KDC16-2),제2019-18호
137,포도과피 효소발효추출물(KL-GEFE)(제2019-27호),포도과피 효소발효추출물(KL-GEFE),제2019-27호
138,MS-10 엉겅퀴등복합추출물(제2020-6호),MS-10 엉겅퀴등복합추출물,제2020-6호
182,인동덩굴꽃봉오리추출물(그린세라-F)(제2019-14호),인동덩굴꽃봉오리추출물(그린세라-F),제2019-14호
194,보리 베타글루칸 추출물(Barley β-glucan Extract)(제2009-73호),보리 베타글루칸 추출물(Barley β-glucan Extract),제2009-73호


In [321]:
i_df.loc[i_df['ingredient_name'].str.contains(r'\·'), check_columns] # '·' 는 구분자로 그대로 사용

,ingredient_raw_name,ingredient_name,ingredient_auth_num
384,옥수수수염·레몬밤추출복합물(CL-5MoA)(제2023-7호),옥수수수염·레몬밤추출복합물(CL-5MoA),제2023-7호
385,옥수수수염·레몬밤추출복합물(CL-5MoA)(제2023-8호),옥수수수염·레몬밤추출복합물(CL-5MoA),제2023-8호
391,참당귀·녹용·황기 복합추출물(제2023-23호),참당귀·녹용·황기 복합추출물,제2023-23호


In [322]:
i_df.loc[i_df['ingredient_name'].str.contains(r'\β'), check_columns] # 'β' 는 의미있는 기호로 그대로 사용

,ingredient_raw_name,ingredient_name,ingredient_auth_num
194,보리 베타글루칸 추출물(Barley β-glucan Extract)(제2009-73호),보리 베타글루칸 추출물(Barley β-glucan Extract),제2009-73호


### 4-3. `ingredient_name` 컬럼의 특수문자 처리

In [323]:
# '®'  => 'Ⓡ'로 대체
i_df['ingredient_name'] = i_df['ingredient_name'].apply(lambda x: re.sub(r'®', 'Ⓡ', x))

# 꺽쇠기호 => 소괄호로 대체
i_df['ingredient_name'] = i_df['ingredient_name'].apply(lambda x: re.sub(r'\<', '(', x))
i_df['ingredient_name'] = i_df['ingredient_name'].apply(lambda x: re.sub(r'\>', ')', x))

# '․'  => '·'로 대체
i_df['ingredient_name'] = i_df['ingredient_name'].apply(lambda x: re.sub(r'\․', '\·', x))

# '뮤코다당.단백' => '뮤코다당·단백'
i_df.loc[i_df['ingredient_name'].str.contains('뮤코다당'), 'ingredient_name'] = '뮤코다당·단백'

# '.' => '.'로 대체
i_df['ingredient_name'] = i_df['ingredient_name'].apply(lambda x: re.sub(r'\.', '.', x))

In [324]:
# 대체된 특수문자 제거된 것을 확인
i_df.loc[i_df['ingredient_name'].str.contains(r'[®\<\>\․]'), check_columns]

,ingredient_raw_name,ingredient_name,ingredient_auth_num


### 4-4. `ingredient_grp_name` 컬럼 생성
- `ingredient_name` 컬럼 값에서 비슷한 영양성분을 그룹으로 묶어 준 컬럼 생성
- 기본적으로 소괄호(`()`)안에 세부정보 포함
- 예외 사항
  - 소괄호와 대괄호와 안의 모든 문자열 : `'리스펙타(Respecta®)[프로바이오틱스 등 복합물](제2019-26호)'`
- 영양성분 추천 리포트 관련 프론트에서 보여줄 예정

#### `ingredient_grp_name` 예외사항 처리(하드코딩)
- 아래 키워드 포함시 예외처리

|`ingredient_grp_name` 컬럼 값에 포함된 키워드 | 키워드 포함시 `ingredient_grp_name` 컬럼 대체 값|
|---|---|
|'자일로올리고당'|'자일로올리고당'|
|'프로바이오틱스'|'프로바이오틱스'|
|'루테인지아자틴'|'루테인/지아자틴'|
|'루테인/지아자틴'|'루테인/지아자틴'|
|'오메가-3'|'오메가3'|
|'밀크씨슬'|'밀크씨슬'|
|'셀레늄'|'셀레늄/셀렌'|
|'Green Mate Extract'|'그린마테 추출물'|

- '자일로올리고당'의 경우 액상, 분말로 구분된 같은 업체에서 만든 제품
  - 자일로올리고당(xylooligosaccharide)(제2009-81호) => 액상
  - 자일로올리고당(xylooligosaccharide) 분말(제2009-80호)
- 나머지는 사용자 편의상 프론트 상에서 더 적절한 용어로 대체

In [325]:
# 'ingredient_grp_name' 컬럼 예외사항 - 추후 업데이트할 dict
update_ingredient_grp_name_dict = {'자일로올리고당' : '자일로올리고당',
                                   '프로바이오틱스' : '프로바이오틱스',
                                   '루테인지아잔틴' : '루테인/지아자틴',
                                   '루테인/지아잔틴' : '루테인/지아자틴',
                                   '오메가-3' : '오메가3',
                                   '밀크씨슬' : '밀크씨슬',
                                   '셀레늄' : '셀레늄/셀렌',
                                   'Green Mate Extract' : '그린마테 추출물'
                                   }

In [326]:
# `ingredient_grp_name` 컬럼 생성하는 함수 정의 
def create_ingredient_grp_name(df, origin_name_col, grp_name_col, cleaned_name_col=None):
    bracket_strs = []
    grp_names = []
    for name in df[origin_name_col]:
        grp_name_mask = r'\([^)]+\)' # 소괄호와 그 안의 모든 문자열
        if re.search(r'\[[^\]]+\]', name):
            grp_name_mask = r'\([^)]+\)\[[^\]]+\]' # 소괄호와 대괄호와 안의 모든 문자열
        # elif re.search(r'\<[^\>]+\>', name):
        #     grp_name_mask = r'\<[^\>]+\>' # 꺽쇠 기호 안의 모든 문자열 - `ingrediet_name` 처리에서 소괄호로 대체함
        
        grp_name = re.sub(grp_name_mask, '', name)
        grp_name = grp_name.strip() # 앞 뒤 공백 삭제
        grp_names.append(grp_name)
        
        if cleaned_name_col is not None:     
            bracket_str = re.search(grp_name_mask, name) 
            if bracket_str:
                bracket_strs.append(bracket_str.group())
            else:
                bracket_strs.append(np.nan)
    
    df[grp_name_col] = grp_names
    
    if cleaned_name_col is not None:        
        df[cleaned_name_col] = bracket_strs
        
    for key, value in update_ingredient_grp_name_dict.items():
        df.loc[df[grp_name_col].str.contains(key), grp_name_col] = value
    
    return df

In [327]:
# 함수 적용
i_df = create_ingredient_grp_name(i_df, 'ingredient_name', 'ingredient_grp_name', 'ingredinet_bracket_str')

# 확인
check_columns_2 = ['ingredient_raw_name', 'ingredient_name', 'ingredient_grp_name', 'ingredinet_bracket_str', 'ingredient_auth_num']
i_df.loc[(i_df['ingredinet_bracket_str'].notnull()), check_columns_2].sort_values(by='ingredient_grp_name')

,ingredient_raw_name,ingredient_name,ingredient_grp_name,ingredinet_bracket_str,ingredient_auth_num
354,B.breve IDCC 4401(BBR4401) 열처리배양건조물(제2021-5호),B.breve IDCC 4401(BBR4401) 열처리배양건조물,B.breve IDCC 4401 열처리배양건조물,(BBR4401),제2021-5호
139,CJ식물성유지 디글리세라이드(DG)(제2009-21호),CJ식물성유지 디글리세라이드(DG),CJ식물성유지 디글리세라이드,(DG),제2009-21호
165,CMO 함유 FAC(Fatty Acid Complex)(제2014-26호),CMO 함유 FAC(Fatty Acid Complex),CMO 함유 FAC,(Fatty Acid Complex),제2014-26호
258,L.plantarum IM76과 B.longum IM55 복합물(NVP1703)(제2019-28호),L.plantarum IM76과 B.longum IM55 복합물(NVP1703),L.plantarum IM76과 B.longum IM55 복합물,(NVP1703),제2019-28호
38,"NAG(엔에이지, N-아세틸글루코사민, N-Acetylglucosamine)","NAG(엔에이지, N-아세틸글루코사민, N-Acetylglucosamine)",NAG,"(엔에이지, N-아세틸글루코사민, N-Acetylglucosamine)",NaN
...,...,...,...,...,...
197,해태올리고펩티드(NORI-PEPTIDE)(제2009-87호),해태올리고펩티드(NORI-PEPTIDE),해태올리고펩티드,(NORI-PEPTIDE),제2009-87호
187,호로파종자추출물(TestofenⓇ)(제2019-15호),호로파종자추출물(TestofenⓇ),호로파종자추출물,(TestofenⓇ),제2019-15호
361,홍삼오일(KGC11o)(제2022-33호),홍삼오일(KGC11o),홍삼오일,(KGC11o),제2022-33호
179,황기추출물 등 복합물(HT042)(제2014-40호),황기추출물 등 복합물(HT042),황기추출물 등 복합물,(HT042),제2014-40호


In [328]:
# 대괄호 기호 포함시 확인
i_df.loc[i_df['ingredient_raw_name'].str.contains(r'\['), check_columns_2]

,ingredient_raw_name,ingredient_name,ingredient_grp_name,ingredinet_bracket_str,ingredient_auth_num
324,리스펙타(Respecta®)[프로바이오틱스 등 복합물](제2019-26호),리스펙타(RespectaⓇ)[프로바이오틱스 등 복합물],리스펙타,(RespectaⓇ)[프로바이오틱스 등 복합물],제2019-26호


In [329]:
# 예외사항 처리 확인
grp_name_mask_a = i_df['ingredient_grp_name'].str.contains('자일로올리고당')
grp_name_mask_b = i_df['ingredient_grp_name'].str.contains('프로바이오틱스')
grp_name_mask_c = i_df['ingredient_grp_name'].str.contains('루테인')
grp_name_mask_d = i_df['ingredient_grp_name'].str.contains('오메가3')
grp_name_mask_e = i_df['ingredient_grp_name'].str.contains('밀크씨슬')
grp_name_mask_f = i_df['ingredient_grp_name'].str.contains('셀레늄')
grp_name_mask_g = i_df['ingredient_grp_name'].str.contains('그린마테')
grp_name_mask = grp_name_mask_a | grp_name_mask_b | grp_name_mask_c | grp_name_mask_d | grp_name_mask_e | grp_name_mask_f | grp_name_mask_g

i_df.loc[grp_name_mask, check_columns_2].sort_values(by='ingredient_grp_name')

,ingredient_raw_name,ingredient_name,ingredient_grp_name,ingredinet_bracket_str,ingredient_auth_num
173,Green Mate Extract EFLA® 920(제2008-52호),Green Mate Extract EFLAⓇ 920,그린마테 추출물,NaN,제2008-52호
305,Green Mate Extract EFLA 920(제2010-7호),Green Mate Extract EFLA 920,그린마테 추출물,NaN,제2010-7호
339,루테인지아잔틴복합추출물(제2018-4호),루테인지아잔틴복합추출물,루테인/지아자틴,NaN,제2018-4호
96,루테인/지아잔틴 복합추출물(제2008-66호),루테인/지아잔틴 복합추출물,루테인/지아자틴,NaN,제2008-66호
362,루테인지아잔틴추출복합물(제2021-21호),루테인지아잔틴추출복합물,루테인/지아자틴,NaN,제2021-21호
234,루테인지아잔틴복합추출물20%(제2013-23호),루테인지아잔틴복합추출물20%,루테인/지아자틴,NaN,제2013-23호
353,루테인지아잔틴복합추출물(제2018-11호),루테인지아잔틴복합추출물,루테인/지아자틴,NaN,제2018-11호
349,루테인지아잔틴추출복합물(제2022-29호),루테인지아잔틴추출복합물,루테인/지아자틴,NaN,제2022-29호
416,루테인지아잔틴복합추출물(제2019-16호),루테인지아잔틴복합추출물,루테인/지아자틴,NaN,제2019-16호
316,밀크씨슬(카르두스 마리아누스) 추출물,밀크씨슬(카르두스 마리아누스) 추출물,밀크씨슬,(카르두스 마리아누스),NaN


### 4-5. `ingredient_recom_name` 컬럼 생성
- 추천시스템 및 영양제 데이터셋의 포함된 영양성분 명 확인용
- 영양성분 타입 관련된 영양DB 데이터셋과 영양성분 명 확인용

In [330]:
# 'ingredient_grp_name'에 포함된 특수문자 및 공백문자 제거 - 'ingredient_recom_name' 컬럼에 추가
i_df['ingredient_recom_name'] = i_df['ingredient_grp_name'].apply(lambda x: re.sub(r'[^가-힣A-Za-z0-9]', '', x))

# 특수문자 및 공백문자 제거되었는지 확인
check_columns_3 = ['ingredient_raw_name', 'ingredient_name', 'ingredient_grp_name', 'ingredinet_bracket_str', 'ingredient_auth_num', 'ingredient_recom_name']
i_df.loc[i_df['ingredient_name'].str.contains(r'[^가-힣A-Za-z0-9]'), check_columns_3]

,ingredient_raw_name,ingredient_name,ingredient_grp_name,ingredinet_bracket_str,ingredient_auth_num,ingredient_recom_name
2,비타민 B6,비타민 B6,비타민 B6,NaN,NaN,비타민B6
7,옥타코사놀 함유 유지,옥타코사놀 함유 유지,옥타코사놀 함유 유지,NaN,NaN,옥타코사놀함유유지
12,식물스테롤/식물스테롤에스테르,식물스테롤/식물스테롤에스테르,식물스테롤/식물스테롤에스테르,NaN,NaN,식물스테롤식물스테롤에스테르
17,L-글루타민산 유래 GABA 함유 분말(제2009-1호),L-글루타민산 유래 GABA 함유 분말,L-글루타민산 유래 GABA 함유 분말,NaN,제2009-1호,L글루타민산유래GABA함유분말
18,중쇄지방산(MCFA) 함유 유지(제2009-20호),중쇄지방산(MCFA) 함유 유지,중쇄지방산 함유 유지,(MCFA),제2009-20호,중쇄지방산함유유지
...,...,...,...,...,...,...
429,녹용효소분해추출물(HENKIVⓇ)(제2024-2호),녹용효소분해추출물(HENKIVⓇ),녹용효소분해추출물,(HENKIVⓇ),제2024-2호,녹용효소분해추출물
430,난각막가수분해물(NEM®)(제2024-9호),난각막가수분해물(NEMⓇ),난각막가수분해물,(NEMⓇ),제2024-9호,난각막가수분해물
431,백편두추출분말(NOVAponinⓇ)(제2024-6호),백편두추출분말(NOVAponinⓇ),백편두추출분말,(NOVAponinⓇ),제2024-6호,백편두추출분말
433,풋사과 농축분말(제2024-4호),풋사과 농축분말,풋사과 농축분말,NaN,제2024-4호,풋사과농축분말


### 4-6. `ingredient_grp_name` 비슷한 성분 예외 처리
- 공백문자 유무로 비슷한 성분이지만 다르게 나타나는 경우 동일한 값으로 처리
- `ingredeint_recom_name`은 중복되지만, `ingredient_grp_name`의 공백문자 차이로 중복되지 않은 경우

In [331]:
# 중복된 ingredient_recom_name을 그룹화 => 각 그룹 내에서 ingredient_grp_name의 unique 값의 개수 확인
duplicate_recom_names = i_df.groupby('ingredient_recom_name')['ingredient_grp_name'].nunique() # unuque() : 지정된 axis에서 고유한 값(unique value)의 개수
duplicate_recom_names = duplicate_recom_names[duplicate_recom_names > 1].index

# 해당 조건을 만족하는 행만 조회
i_df.loc[i_df['ingredient_recom_name'].isin(duplicate_recom_names), check_columns_3].sort_values(by='ingredient_grp_name')

,ingredient_raw_name,ingredient_name,ingredient_grp_name,ingredinet_bracket_str,ingredient_auth_num,ingredient_recom_name
174,Nopal 추출물(제2009-39호),Nopal 추출물,Nopal 추출물,NaN,제2009-39호,Nopal추출물
23,Nopal추출물(제2010-10호),Nopal추출물,Nopal추출물,NaN,제2010-10호,Nopal추출물
236,강황 추출물(터마신)(제2014-2호),강황 추출물(터마신),강황 추출물,(터마신),제2014-2호,강황추출물
376,강황추출물(제2023-5호),강황추출물,강황추출물,NaN,제2023-5호,강황추출물
195,루바브뿌리 추출물(제2019-6호),루바브뿌리 추출물,루바브뿌리 추출물,NaN,제2019-6호,루바브뿌리추출물
365,루바브뿌리추출물(제2022-28호),루바브뿌리추출물,루바브뿌리추출물,NaN,제2022-28호,루바브뿌리추출물
334,루바브뿌리추출물(제2021-17호),루바브뿌리추출물,루바브뿌리추출물,NaN,제2021-17호,루바브뿌리추출물
105,보스웰리아 추출물(Boswellia serrata R. extract)(제2014-23호),보스웰리아 추출물(Boswellia serrata R. extract),보스웰리아 추출물,(Boswellia serrata R. extract),제2014-23호,보스웰리아추출물
342,보스웰리아추출물(SERRATRIN)(제2022-17호),보스웰리아추출물(SERRATRIN),보스웰리아추출물,(SERRATRIN),제2022-17호,보스웰리아추출물
175,보이차 추출물(제2009-17호),보이차 추출물,보이차 추출물,NaN,제2009-17호,보이차추출물


In [332]:
def update_sim_grp_name(df, recom_name_col, grp_name_col):
    # 중복된 추천 이름 그룹화 및 유일한 그룹 이름 찾기
    duplicate_recom_names = df.groupby(recom_name_col)[grp_name_col].nunique()
    duplicate_recom_names = duplicate_recom_names[duplicate_recom_names > 1].index
    
    check_dict = {}
    
    for recom_name in duplicate_recom_names:
        # 중복된 추천 이름에 대해 그룹 이름 리스트 가져오기
        grp_name_list = df.loc[df[recom_name_col] == recom_name, grp_name_col].tolist()
        
        # 가장 긴 문자열의 길이 찾기
        max_length = max(len(name) for name in grp_name_list)
        
        # 가장 긴 문자열의 길이와 동일한 길이를 가진 문자열들만 남기고 문자열로 변환
        grp_name_list = [name for name in grp_name_list if len(name) == max_length]
        unique_grp_name = list(set(grp_name_list))[0]  # 중복된 값 제거 후 첫 번째 문자열 선택
        
        check_dict[recom_name] = unique_grp_name
    
    # check_dict를 이용해서, grp_name_col 값 업데이트
    for key, value in check_dict.items():
        df.loc[df[recom_name_col] == key, grp_name_col] = value
    
    return df

In [333]:
i_df = update_sim_grp_name(i_df, 'ingredient_recom_name', 'ingredient_grp_name')

i_df.loc[i_df['ingredient_recom_name'].isin(duplicate_recom_names), check_columns_3].sort_values(by='ingredient_grp_name')

,ingredient_raw_name,ingredient_name,ingredient_grp_name,ingredinet_bracket_str,ingredient_auth_num,ingredient_recom_name
23,Nopal추출물(제2010-10호),Nopal추출물,Nopal 추출물,NaN,제2010-10호,Nopal추출물
174,Nopal 추출물(제2009-39호),Nopal 추출물,Nopal 추출물,NaN,제2009-39호,Nopal추출물
376,강황추출물(제2023-5호),강황추출물,강황 추출물,NaN,제2023-5호,강황추출물
236,강황 추출물(터마신)(제2014-2호),강황 추출물(터마신),강황 추출물,(터마신),제2014-2호,강황추출물
365,루바브뿌리추출물(제2022-28호),루바브뿌리추출물,루바브뿌리 추출물,NaN,제2022-28호,루바브뿌리추출물
334,루바브뿌리추출물(제2021-17호),루바브뿌리추출물,루바브뿌리 추출물,NaN,제2021-17호,루바브뿌리추출물
195,루바브뿌리 추출물(제2019-6호),루바브뿌리 추출물,루바브뿌리 추출물,NaN,제2019-6호,루바브뿌리추출물
342,보스웰리아추출물(SERRATRIN)(제2022-17호),보스웰리아추출물(SERRATRIN),보스웰리아 추출물,(SERRATRIN),제2022-17호,보스웰리아추출물
105,보스웰리아 추출물(Boswellia serrata R. extract)(제2014-23호),보스웰리아 추출물(Boswellia serrata R. extract),보스웰리아 추출물,(Boswellia serrata R. extract),제2014-23호,보스웰리아추출물
175,보이차 추출물(제2009-17호),보이차 추출물,보이차 추출물,NaN,제2009-17호,보이차추출물


## 5. `ingredient_type` 영양성분 타입 컬럼 추가

### 5-1. 영양소
- Open API 제공 관련 사이트에 업데이트 된 내용 참고해 하드코딩(총 28개)
  - [식품안전나라 사이트 건강기능식품 원료별 정보 '영양성분' 카테고리](https://www.foodsafetykorea.go.kr/portal/board/board.do?menu_grp=MENU_NEW01&menu_no=2660)
  - [건강기능식품 정보포털 사이트 - 건강기능식품 검색 - 원료 검색 - 영양소(고시형) 카테고리](https://www.hffinfo.com/search/raw_material?page=1&type=0)

- '비타민' or '셀레늄' 키워드 보유시
- '베타카로틴', '나이아신', '판토텐산', '엽산', '비오틴', '칼슘', '마그네슘', '철', '아연', '구리', '요오드', '망간', '몰리브덴', '칼륨', '크롬', '식이섬유', '필수지방산', '필수 지방산', '단백질'

In [334]:
i_df['ingredient_type'] = np.nan

# 'ingredient_type'컬럼에 '영양소'의 값을 가지는 영양성분명
check_ingredient_names = ['베타카로틴', '나이아신', '판토텐산', '엽산', '비오틴', '칼슘', '마그네슘', '철', '아연', '구리', '요오드', '망간', '몰리브덴', '칼륨', '크롬', '식이섬유', '필수지방산', '필수 지방산', '단백질']

# 'ingredient_name'컬럼값이 check_ingredient_names 리스트에 포함되어 있으면 'ingredient_type'컬럼값은 '영양소'
i_df.loc[(i_df['ingredient_name'].isin(check_ingredient_names)) | (i_df['ingredient_name'].str.contains('셀레늄')), 'ingredient_type'] = '영양소'

# 'ingredient_name'컬럼값에 '비타민' or '셀레늄' 키워드 보유시 'ingredient_type'컬럼값은 '영양소'
i_df.loc[(i_df['ingredient_name'].str.contains('비타민')) | (i_df['ingredient_name'].str.contains('셀레늄')), 'ingredient_type'] = '영양소'

### 5-2. 개별인정원료
- `ingredient_auth_num`이 결측치가 아닌 경우 영양성분 타입은 '개별인정원료'이다.

In [335]:
# `ingredient_auth_num`이 결측치가 아닌 경우 'ingredient_type' 컬럼값은 '개별인정원료'
i_df.loc[i_df['ingredient_auth_num'].notnull(), ['ingredient_type']] = '개별인정원료'

In [336]:
# 확인
check_columns_5 = ['ingredient_name', 'ingredient_grp_name', 'ingredient_auth_num', 'ingredient_recom_name', 'ingredient_type']
i_df.loc[i_df['ingredient_auth_num'].notnull(), check_columns_5 ]

,ingredient_name,ingredient_grp_name,ingredient_auth_num,ingredient_recom_name,ingredient_type
14,커피만노올리고당분말,커피만노올리고당분말,제2009-15호,커피만노올리고당분말,개별인정원료
15,식물스타놀에스테르,식물스타놀 에스테르,제2006-11호,식물스타놀에스테르,개별인정원료
16,석류추출물,석류추출물,제2010-23호,석류추출물,개별인정원료
17,L-글루타민산 유래 GABA 함유 분말,L-글루타민산 유래 GABA 함유 분말,제2009-1호,L글루타민산유래GABA함유분말,개별인정원료
18,중쇄지방산(MCFA) 함유 유지,중쇄지방산 함유 유지,제2009-20호,중쇄지방산함유유지,개별인정원료
...,...,...,...,...,...
431,백편두추출분말(NOVAponinⓇ),백편두추출분말,제2024-6호,백편두추출분말,개별인정원료
432,발효굴추출물,발효굴추출물,제2023-41호,발효굴추출물,개별인정원료
433,풋사과 농축분말,풋사과 농축분말,제2024-4호,풋사과농축분말,개별인정원료
434,기장밀추출복합물(KeranatTM),기장밀추출복합물,제2024-16호,기장밀추출복합물,개별인정원료


### 5-3. 기능성원료
- `ingredient_type` 컬럼값으로 '영양소' 또는 '개별인정원료'로 분류되지 않고 결측치일 경우 '기능성원료'로 대체
- Open API 제공 관련 사이트에 공시된 기능성원료(69개)와 일치

In [337]:
i_df.loc[(i_df['ingredient_type'].isnull()), check_columns_5] # 69개 확인

,ingredient_name,ingredient_grp_name,ingredient_auth_num,ingredient_recom_name,ingredient_type
7,옥타코사놀 함유 유지,옥타코사놀 함유 유지,NaN,옥타코사놀함유유지,NaN
8,홍삼,홍삼,NaN,홍삼,NaN
9,매실추출물,매실추출물,NaN,매실추출물,NaN
10,대두단백,대두단백,NaN,대두단백,NaN
11,클로렐라,클로렐라,NaN,클로렐라,NaN
...,...,...,...,...,...
411,프로폴리스추출물,프로폴리스추출물,NaN,프로폴리스추출물,NaN
412,홍국,홍국,NaN,홍국,NaN
413,차전자피식이섬유,차전자피식이섬유,NaN,차전자피식이섬유,NaN
414,감마리놀렌산 함유 유지,감마리놀렌산 함유 유지,NaN,감마리놀렌산함유유지,NaN


In [338]:
# `ingredient_type` 값이 결측치일 경우 '기능성원료'로 값 대체
i_df.fillna(value={'ingredient_type' : '기능성원료'}, inplace=True)

# 'ingredient_type'에 결측치 없는 것을 확인
i_df[i_df['ingredient_type'].isnull()]

,ingredient_raw_name,ingredient_function,ingredient_caution,ingredient_limit_high,ingredient_limit_low,ingredeint_unit,ingredient_etc,ingredient_auth_num,ingredient_name,ingredient_grp_name,ingredinet_bracket_str,ingredient_recom_name,ingredient_type


### 5-4. 영양성분 타입 컬럼 생성 함수화
- 위 내용을 바탕으로 함수화

In [339]:
# 'ingredient_type'컬럼에 '영양소'의 값을 가지는 영양성분명
check_ingredient_names = ['베타카로틴', '나이아신', '판토텐산', '엽산', '비오틴', '칼슘', '마그네슘', '철', '아연', '구리', '요오드', '망간', '몰리브덴', '칼륨', '크롬', '식이섬유', '필수지방산', '필수 지방산', '단백질']

In [340]:
# 영양성분 타입 생성하는 함수 정의
def create_ingredient_type(df, cleaned_name_col, auth_num_col, type_col):
    global check_ingredient_names
    # 영양성분 타입 컬럼 추가 - 결측치로 값 넣기
    df[type_col] = np.nan
    
    # 영양성분 명이 check_ingredient_names 리스트에 포함되어 있으면 '영양소'
    df.loc[(df[cleaned_name_col].isin(check_ingredient_names)) | (df[cleaned_name_col].str.contains('셀레늄')), type_col] = '영양소'

    # 영양성분 명에 '비타민' or '셀레늄' 키워드 보유시 '영양소'
    df.loc[(df[cleaned_name_col].str.contains('비타민')) | (df[cleaned_name_col].str.contains('셀레늄')), type_col] = '영양소'

    # 개별인정원료 인정번호가 결측치가 아닌 경우 '개별인정원료'
    df.loc[df[auth_num_col].notnull(), [type_col]] = '개별인정원료'

    # 위 처리 후 에도 결측치일 경우 '기능성원료'
    df.fillna(value={type_col : '기능성원료'}, inplace=True)
    return df      

In [341]:
# 영양성분 타입 컬럼 생성하는 함수 적용
i_df = create_ingredient_type(i_df, 'ingredient_name', 'ingredient_auth_num', 'ingredient_type')

# 'ingredient_type'에 결측치 없는 것을 확인
i_df[i_df['ingredient_type'].isnull()]

,ingredient_raw_name,ingredient_function,ingredient_caution,ingredient_limit_high,ingredient_limit_low,ingredeint_unit,ingredient_etc,ingredient_auth_num,ingredient_name,ingredient_grp_name,ingredinet_bracket_str,ingredient_recom_name,ingredient_type


In [342]:
# 이전 작업 버전에서 잘 못 분류된 영양성분 타입 재확인. ingredient_type이 key값으로 존재해야한다.
updated_type_dict = {'기능성원료' : ['유단백가수분해물', '귀리식이섬유', '폴리감마글루탐산'], 
                     '개별인정원료' : ['루테인지아잔틴복합추출물(제2018-4호)', '마리골드 추출물(루테인 에스테르)(제2012-22호)', '호로파종자추출물(TestofenⓇ)(제2019-15호)'] 
                     }

# 확인하니 제대로 분류된 것을 확인
for key, values in updated_type_dict.items():
    for value in values:
        print(i_df.loc[i_df['ingredient_raw_name'] == value, ['ingredient_raw_name', 'ingredient_type']])

   ingredient_raw_name ingredient_type
55            유단백가수분해물           기능성원료
   ingredient_raw_name ingredient_type
79              귀리식이섬유           기능성원료
   ingredient_raw_name ingredient_type
40            폴리감마글루탐산           기능성원료
        ingredient_raw_name ingredient_type
339  루테인지아잔틴복합추출물(제2018-4호)          개별인정원료
               ingredient_raw_name ingredient_type
248  마리골드 추출물(루테인 에스테르)(제2012-22호)          개별인정원료
                ingredient_raw_name ingredient_type
187  호로파종자추출물(TestofenⓇ)(제2019-15호)          개별인정원료


- 모두 맞게 분류된 것을 확인

In [343]:
i_df['ingredient_type'].value_counts()

개별인정원료    339
기능성원료      69
영양소        28
Name: ingredient_type, dtype: int64

## 6. 영양성분 섭취량 컬럼 전처리
- 섭취량을 나타내는 `ingredeint_limit_high`, `ingredient_limit_low` 컬럼은 `int` 혹은 `float` 형태로 보이지만 문자열로 되어 있다.
- 숫자와 `.` 이외의 문자열이 들어가있는지 확인

In [344]:
i_df.loc[i_df['ingredient_limit_high'].str.contains(r'[^0-9.]'), 'ingredient_limit_high'] # 숫자, '.' 문자열로만 구성된 것으로 확인

Series([], Name: ingredient_limit_high, dtype: object)

In [345]:
i_df.loc[i_df['ingredient_limit_low'].str.contains(r'[^0-9.]'), 'ingredient_limit_low'] # 숫자, '.' 문자열로만 구성된 것으로 확인

Series([], Name: ingredient_limit_low, dtype: object)

## 7.건강기능 및 주의사항 컬럼 분석 및 전처리

### 7-1. 건강기능 및 주의사항 컬럼 전처리 내용 분석
- 'HF00', 'HF01', ... , 'HF26' 총 27개의 건강기능 코드로 분류하기 위해 `ingredient_function` 컬럼 전처리를 위한 내용 확인
- 주의사항 코드로 분류하기 위해 `ingredient_caution` 컬럼 전처리를 위한 내용 확인
  - 기저질환 및 관련 복용 의약품 코드, 알레르기 및 민감성분 코드, 수술 여부 코드, 임신 및 수유 여부 코드, 연령대 코드, 성별 코드, 흡연 여부 코드와 관련

In [346]:
# 전처리가 필요한 특수문자 확인 - 함수 활용
function_special_chars = find_special_chars(i_df, 'ingredient_function')
caution_special_chars = find_special_chars(i_df, 'ingredient_caution')

# 두 컬럼에 있는 특수문자 리스트 합치기
special_cahrs = function_special_chars + caution_special_chars

# 리스트 내 중복된 문자열 제거
special_cahrs = sorted(list(set(special_cahrs)))

print(special_cahrs)

['(', ')', '+', ',', '-', '.', ':', '`', '·', 'β', '‘', '’', '․', '‧', '※', 'Ⅱ', '①', '②', '③', '④', '⑤', 'ⓛ', '・', '･']


In [347]:
# 특정 문자(열)가 들어가있는 데이터프레임 생성하는 함수정의
def find_chars_df(df, col_name_list, check_chars):
    check_chars_df = pd.DataFrame(columns=df.columns) # 결과를 저장할 데이터프레임
    
    for col_name in col_name_list:
        for char in check_chars:
            mask_1 = df[col_name].notnull()
            mask_2 = df[col_name].str.contains(char, regex=False) # 특수 문자를 처리하기 위해 `regex=False` 사용
             
            if mask_2.any():
                df2 = df[(mask_1) & (mask_2)]
                check_chars_df = pd.concat([check_chars_df, df2], axis=0, ignore_index=True)
            else:
                print(f'{col_name} 컬럼값에 "{char}" 문자열이 없습니다.')
            
    # 중복 제거
    check_chars_df.drop_duplicates(subset=col_name_list, inplace=True, ignore_index=True)
    
    return check_chars_df

In [348]:
check_function_df = create_ingredient_grp_name(i_df, 'ingredient_function', 'ingredient_function_content', 'function_bracket_str')
check_function_df.loc[check_function_df['function_bracket_str'].notnull(), ['ingredient_function', 'ingredient_function_content', 'function_bracket_str']]
check_function_df.loc[check_function_df['function_bracket_str'].notnull(), ['function_bracket_str']].value_counts()

function_bracket_str
(국문)                    218
(생리활성기능 2등급)             59
(기타기능II)                 28
(1)                      12
(기타기능 II)                 7
(기타기능Ⅱ)                   7
(기타기능 Ⅱ)                  2
(Diacylglyceride)         1
(기타 II)                   1
(기타Ⅱ)                     1
(내중막 두께 : IMT)            1
(생리활성기능 2등급I)             1
(생리활성기능2등급)               1
(코 가려움, 재채기, 콧물)          1
dtype: int64

- 기능성 원료의 기능성 인정등급(출처 : 건강기능식품 식약처)
  
|기능성 등급|기능성 내용|
|---|---|
|질병발생 위험 감소 기능|OO발생 위험 감소에 도움을 줌|
|생리활성기능 1 등급|OO에 도움을 줌|
|생리활성기능 2 등급|OO에 도움을 줄 수 있음|
|생리활성기능 3 등급|OO에 도움을 줄 수 있으나 관련 인체적용시험이 미흡함|

In [349]:
find_chars_df(i_df, ['ingredient_function'], ['생리활성기능'])[['ingredient_function']]

,ingredient_function
0,이 제품은 중쇄지방산을 함유하고 있어 다른 식용유와 비교하였을 때 체지방 증가가 적을 수 있음 (생리활성기능 2등급)
1,요로의 유해균 흡착 억제로 요로건강에 도움을 줄 수 있음(생리활성기능 2등급)
2,피부보습에 도움을 줄 수 있음(생리활성기능 2등급)
3,면역기능 증진에 도움을 줄 수 있음(생리활성기능 2등급)
4,콜레스테롤 개선에 도움을 줄 수 있음 (생리활성기능 2등급)
5,관절 건강에 도움을 줄 수 있음(생리활성기능 2등급)
6,(국문) 지구성 운동 시 피로개선에 도움을 줄 수 있음 (생리활성기능 2등급)\n(영문) May help to relieve fatigue recovery
7,(국문) 노인의 기억력 개선에 도움을 줄 수 있음 (생리활성기능 2등급)\n(영문) May help to improve abilities to remember
8,관절 및 연골 건강에 도움을 줄 수 있음(생리활성기능 2등급)
9,식후 혈당상승 억제에 도움을 줄 수 있음 (생리활성기능 2등급)


In [350]:
find_chars_df(i_df, ['ingredient_function'], ['(기타'])[['ingredient_function']]

,ingredient_function
0,"장내 유익균 증식, 유해균 억제, 배변활동에 도움을 줄 수 있음 (기타기능 II)"
1,갱년기 여성의 건강에 도움을 줄 수 있음(기타Ⅱ)
2,혈압이 높은 사람에게 도움을 줄 수 있습니다. (기타기능Ⅱ)
3,이 제품은 중쇄지방산을 함유하고 있어 다른 식용유와 비교하였을 때 체지방 증가가 적을 수 있음(기타기능II)
4,혈압이 높은 사람에게 도움을 줄 수 있습니다(기타기능 II)
5,인체의 항산화능 증진에 도움을 줄 수 있습니다.(기타기능 II)
6,항산화능 증진에 도움을 줄 수 있음(기타기능II)
7,식후 혈당조절에 도움을 줄 수 있습니다 (기타기능II)
8,장내유익균의 증식과 유해균의 억제에 도움을 줄 수 있음(기타기능II)배변활동을 원활히 하는데 도움을 줄 수 있음(기타기능II)
9,혈압이 걱정되시는 분에게 도움을 줄 수 있습니다.(기타기능II)


- 특정 건강기능에 도움을 줄 수 있다는 의미로 생리활성 기능 2등급과 기타 기능 2등급은 유사한 의미로 사용되고 있는 것 같다.
  - 해당 내용은 전처리로 제거해도 무방한 내용으로 보인다.

- `'(기타기능II)'` 또는 `'(기타II 등급)'` 또는 `'(기타 II)'` 또는 `'(기타Ⅱ)'` 또는 `'(생리활성기능)'` 또는`'(생리활성기능2등급)'`이라는 문자가 있을 경우
  - 기능에 대한 문장 구분자로도 쓰이기 때문에 `'\n'`으로 대체

In [351]:
find_chars_df(i_df, ['ingredient_function'], ['(국문)', '(영문)', '[국문]', '[영문]'])[['ingredient_function']]

ingredient_function 컬럼값에 "[국문]" 문자열이 없습니다.
ingredient_function 컬럼값에 "[영문]" 문자열이 없습니다.


,ingredient_function
0,(국문) 체지방 감소에 도움을 줄 수 있음\n(영문) May help to reduce body fat
1,-(국문) 면역기능 증진에 도움을 줄 수 있음\n (영문) May help to improve immune function
2,(국문) 관절건강에 도움을 줄 수 있음\n(영문) May help to maintain healthy joint
3,- (국문) 체지방 감소에 도움을 줄 수 있음\n (영문) May help to reduce body fat
4,(국문) 갱년기 여성 건강에 도움을 줄 수 있음 (영문) May help to menopausal womens health
...,...
142,(국문) 노화로 저하된 인지기능 개선에 도움을 줄 수 있음\n(영문) May help to improve cognitive function affected by aging
143,(국문)\n ① 피부 보습에 도움을 줄 수 있음\n ② 자외선에 의한 피부 손상으로부터 피부 건강을 유지하는데 도움을 줄 수 있음\n(영문)\n ① May help to moisturize (dry) skin\n ② May help to maintain good skin health by protecting against damage from UV exposure
144,"(국문) ① 관절 건강에 도움을 줄 수 있음, ② 위 점막을 보호하여 위 건강에 도움을 줄 수 있음\n(영문) ① May help to maintain healthy joint, ② May help to maintain healthy stomach by protecting gastric mucosa"
145,"(국문) ① 피부 보습에 도움을 줄 수 있음, ② 자외선에 의한 피부손상으로부터 피부건강 유지에 도움을 줄 수 있음\n(영문) ① May help to maintain skin moisturizing, ② May help to maintain skin health from skin damage by UV radiation"


- '(국문)', '(영문)'이 포함된 내용은 해당 괄호와 괄호 안의 내용을 제거하고, 영문으로 번역된 문장은 제거해도 무방한 것으로 보인다.

- (국문)' 또는 '[국문]' 또는 '(영문)' 또는 '[영문]'이라는 문자 제거
    - '(영문)' 또는 '[영문]'과 해당 문자열 뒤에 모든 문자열 제거

In [352]:
find_chars_df(i_df, ['ingredient_function'], ['-'])[['ingredient_function']]

,ingredient_function
0,-(국문) 면역기능 증진에 도움을 줄 수 있음\n (영문) May help to improve immune function
1,-장내 유익균의 증식과 유해균의 억제에 도움을 줄 수 있음\n-배변활동을 원활히 하는데 도움을 줄 수 있음
2,-피부건강·장 건강·면역력 증진에 도움을 줄 수 있음
3,-산화·환원 효소의 활성에 필요
4,- (국문) 체지방 감소에 도움을 줄 수 있음\n (영문) May help to reduce body fat
5,(국문)\n -자외선에 의한 피부손상으로부터 피부건강을 유지하는데 도움을 줄 수 있음\n -피부보습에 도움을 줄 수 있음\n(영문)\n -May help to maintain skin health from skin damage by UV radiation\n -May help to maintain skin moisturizing
6,(국문) 알콜성 손상으로부터 간을 보호하는데 도움을 줄 수 있음(기타기능Ⅱ)\n(영문) May help to maintain liver protection from alcohol-induced damage
7,- 혈당 조절에 도움을 줄 수 있음 (생리활성기능 2등급)\n- 혈압이 높은 사람에게 도움을 줄 수 있음 (생리활성기능 2등급)\n- 체지방 감소에 도움을 줄 수 있음 (생리활성기능 2등급)
8,(국문) 혈관이완을 통한 혈행개선에 도움을 줄 수 있음\n(영문) May help to blood flow improvement by vaso-relaxative action
9,-산화스트레스로부터 인체를 보호하는데 도움을 줄 수 있습니다.\n-혈관벽 두께(내중막 두께 : IMT) 증가 억제를 통한 혈행개선에 도움을 줄 수 있음\n-자외선에 의한 피부홍반개선으로 피부건강에 도움을 줄 수 있음


- `'-'` 문자 기호가 구분자로서 사용되고 있고, 단어의 연결자로서도 사용되고 있는 것을 확인

In [353]:
find_chars_df(i_df, ['ingredient_function', 'ingredient_caution'], ['(1)', '(가)'])[['ingredient_function', 'ingredient_caution']]

ingredient_function 컬럼값에 "(가)" 문자열이 없습니다.


,ingredient_function,ingredient_caution
0,(국문)\n(1) 자외선에 의한 피부손상으로부터 피부 건강을 유지하는데 도움을 줄 수 있음\n(2) 체지방 감소에 도움을 줄 수 있음\n(3) 피부보습에 도움을 줄 수 있음\n(영문)\n(1) May help to maintain skin health from skin damage by UV radiation\n(2) May help to reduce body fat\n(3) May help to maintain skin moisturizing,NaN
1,(1) 자외선에 의한 피부손상으로부터 피부 건강을 유지하는데 도움을 줄 수 있음(생리활성기능 2등급)\n(2) 체지방 감소에 도움을 줄 수 있음(생리활성기능 2등급),NaN
2,(1) 정상적인 혈액응고에 필요\n(2) 뼈의 구성에 필요,(1) 항응고제 등 복용 시 전문가와 상담할 것\n(2) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
3,"(1) 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요",(1)
4,"(1) 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요",(1) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
...,...,...
130,①혈중 콜레스테롤 개선·식후 혈당상승 억제·배변활동 원활에 도움을 줄 수 있음,(가) 반드시 충분한 물과 함께 섭취할 것 (액상제외)\n(나) 대두에 알레르기를 나타내는 사람은 섭취에 주의
131,①칼슘과 인이 흡수되고 이용되는데 필요②뼈의 형성과 유지에 필요③골다공증발생 위험 감소에 도움을 줌,(가) 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n(나) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
132,유산균 증식 및 유해균 억제･배변활동 원활･장 건강에 도움을 줄 수 있음,(가) 질환이 있거나 의약품 복용 시 전문가와 상담할 것\n(나) 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n(다) 어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도할 것\n(라) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
133,①배변활동 원활에 도움을 줄 수 있음,"(가) 어린이, 임산부 및 수유부는 섭취를 피할 것\n(나) 위·신장·간질환이 있거나 의약품 복용 시 전문가와 상담할 것\n(다) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"


- '(1)', '(2)' 와 같이 소괄호 안에 숫자(+공백문자)만 있는 경우는 구분자로 사용되었기 때문에 제거하고, 구분자로서 건강기능 구분만 처리해야 한다.

- 괄호(`()`, `[]`)안에 숫자 혹은 가, 나, 다.. 같은 문자열이 들어가 기능 혹은 주의사항의 구분자 역할을 해주는 경우 `'\n'`으로 대체
    - 괄호 안에 공백문자 포함되는 경우 있음 :  ex. `'( 1)'`
    - 괄호 앞에 공백이 있는 경우와 linespace가 나오는 두 가지 경우가 있음 : ex.`' (2)'`, `'\n(2)'`

In [354]:
find_chars_df(i_df, ['ingredient_function'], ['Diacylglyceride', '내중막 두께', '(코 가려움, 재채기, 콧물)'])[['ingredient_function']]

,ingredient_function
0,이 제품은 디글리세라이드(Diacylglyceride)를 함유하고 있어 다른 식용유와 비교했을 때 식후 혈중 중성지방과 체지방 증가가 적을 수 있음(기타기능 II)
1,-산화스트레스로부터 인체를 보호하는데 도움을 줄 수 있습니다.\n-혈관벽 두께(내중막 두께 : IMT) 증가 억제를 통한 혈행개선에 도움을 줄 수 있음\n-자외선에 의한 피부홍반개선으로 피부건강에 도움을 줄 수 있음
2,"과민반응에 의한 코 상태(코 가려움, 재채기, 콧물) 개선에 도움을 줄 수 있음(기타기능 II)"


- 이외의 소괄호 안의 내용은 세부 사항을 설명하는 내용이기 떄문에 제거하지 않는다.

In [355]:
check_caution_df = create_ingredient_grp_name(i_df[i_df['ingredient_caution'].notnull()], 'ingredient_caution', 'ingredient_caution_content', 'caution_bracket_str')
check_caution_df.loc[check_caution_df['caution_bracket_str'].notnull(), ['ingredient_caution', 'ingredient_caution_content', 'caution_bracket_str']]
check_caution_df.loc[check_caution_df['caution_bracket_str'].notnull(), ['caution_bracket_str']].value_counts()

caution_bracket_str       
(1)                           125
(알레르기 체질 등)                    33
(가)                            14
(액상제외)                         12
(해조류, 어패류 등)                    5
(당뇨치료제, 혈액항응고제)                 5
(게 또는 새우를 원재료로 사용한 경우에 한함)      3
(혈액항응고제)                        2
(항응고제)                          2
(옻 알레르기)                        1
(혈당강하제)                         1
(항히스타민제 또는 중추신경계억제제)            1
(항응고제, 항혈소판제제)                  1
(커피, 홍차, 녹차 등)                  1
(1 )                            1
(액상 제외)                         1
(면역억제제 등)                       1
(담낭질환 등)                        1
(구토와 위장운동 증가)                   1
(구토와  위장운동 증가)                  1
(β-carotene)                    1
(알레르기 체질, 간·신장 질환 환자 등)         1
dtype: int64

In [356]:
find_chars_df(i_df, ['ingredient_caution'], ['(가)'])[['ingredient_caution']]

,ingredient_caution
0,"(가) 섭취 시 가스참, 트림, 복통, 복부팽만감 등이 발생할 수 있음\n(나) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
1,"(가) 어린이, 임산부 및 수유부는 섭취를 피할 것\n(나) 간질환이 있거나 의약품 복용 시 전문가와 상담할 것\n(다) 카페인이 함유되어 있어 초조감, 불면 등을 나타낼 수 있음\n(라) 식사 후 섭취할 것\n(마) 카페인을 함유한 식품의 섭취에 주의할 것\n(바) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
2,"(가) 신장에 영향을 미치는 약물을 복용하거나 신장 이상의 위험이 있는 사람은 의사와 상담 후에 섭취하여야 함\n(나) 어린이, 임산부, 수유부는 섭취를 삼가\n(다) 카페인은 크레아틴의 기능을 감소시킬 수 있음\n(라) 크레아틴의 섭취는 탈수를 동반할 수 있으므로 충분한 물과 함께 섭취하여야 함\n(마) 과다 섭취하지 말 것"
3,"(가) 임산부, 수유여성 및 어린이는 섭취에 주의\n(나) 우유 및 유제품에 대하여 알레르기를 나타내는 사람은 섭취에 주의"
4,"(가) 알레르기 체질이신 분은 개인에 따라 과민반응을 나타낼 수 있음\n(나) 과량 섭취 시 피부색이 오렌지색으로 변할 수 있음\n(다) 임산부, 수유 여성 및 어린이는 섭취에 주의"
5,"(가) 어린이나 임산부, 수유부는 섭취를 주의해야 함\n(나) 충분한 물과 함께 섭취해야 함\n(다) 과다 섭취 시 미약한 설사, 구토, 배변량 증가, 배변빈도 증가, 복부 팽만, 두통 등의 부작용을 일으킬 수 있음"
6,"(가) 임산부 및 수유부는 섭취를 피할 것\n(나) 간·심장질환, 수술 전후, 고혈압, 당뇨 및 천식이 있거나 의약품(항응고제 등) 복용 시 전문가와 상담할 것\n(다) 게 또는 새우에 알레르기가 있는 사람은 섭취에 주의할 것(게 또는 새우를 원재료로 사용한 경우)\n(라) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
7,"(가) 성인남성만 섭취할 것\n(나) 수술 전후, 출혈성 질환이 있거나 항응고제 등 복용 시 전문가와 상담할 것\n(다) 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n(라) 메스꺼움 등 소화계통의 불편함과 설사를 유발할 수 있으니 식사 후 섭취할 것\n(마) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
8,"(가) 어린이, 임산부 및 수유부는 섭취를 피할 것\n(나) 간·신장·심장질환, 알레르기 및 천식이 있거나 의약품 복용 시 전문가와 상담할 것\n(다) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
9,(가) 반드시 충분한 물과 함께 섭취할 것 (액상제외)\n(나) 대두에 알레르기를 나타내는 사람은 섭취에 주의


- '(가)', '(나)' 와 같이 소괄호 안에 가, 나, 다, ... ,하(+공백문자)와 같이 구분자 한글을 사용하는 경우 제거하고, 구분자로서 건강기능 구분만 처리해야 한다.
- 이외의 소괄호 안의 내용은 세부 사항을 설명하는 내용이기 떄문에 제거하지 않는다.

In [357]:
find_chars_df(i_df, ['ingredient_function', 'ingredient_caution'], ['+'])[['ingredient_function', 'ingredient_caution']]

ingredient_function 컬럼값에 "+" 문자열이 없습니다.


,ingredient_function,ingredient_caution
0,방광의 배뇨기능 개선에 도움을 줄 수 있음(생리활성기능 2등급),"-영유아, 어린이, 임산부와 수유부는 섭취에 주의\n-호박씨, 대두에 알레르기를 나타내는 사람은 섭취에 주의\n-에스트로겐 호르몬에 민감한 사람은 섭취에 주의+R-45"


- `'+R-45'` : 예외처리로 해당 문자열 자체를 `''`로 대체하여 제거

In [358]:
find_chars_df(i_df, ['ingredient_function', 'ingredient_caution'], [':'])[['ingredient_function', 'ingredient_caution']]

,ingredient_function,ingredient_caution
0,-산화스트레스로부터 인체를 보호하는데 도움을 줄 수 있습니다.\n-혈관벽 두께(내중막 두께 : IMT) 증가 억제를 통한 혈행개선에 도움을 줄 수 있음\n-자외선에 의한 피부홍반개선으로 피부건강에 도움을 줄 수 있음,밀 단백질에 알레르기를 나타내는 사람은 섭취에 주의
1,①피부건강·항산화에 도움을 줄 수 있음,섭취 시 주의사항 : 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
2,항산화･혈중 콜레스테롤 개선에 도움을 줄 수 있음,섭취 시 주의사항 : 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것


- '섭취 시 주의사항 :'의 경우 컬럼의 내용과 중복되기 때문에 제거 필요

In [359]:
find_chars_df(i_df, ['ingredient_function', 'ingredient_caution'], ['·', '‧', '・', '･', '․'])[['ingredient_function', 'ingredient_caution']]

ingredient_function 컬럼값에 "‧" 문자열이 없습니다.
ingredient_function 컬럼값에 "・" 문자열이 없습니다.
ingredient_function 컬럼값에 "․" 문자열이 없습니다.


,ingredient_function,ingredient_caution
0,①면역력 증진·피로개선·혈소판 응집억제를 통한 혈액흐름·기억력 개선·항산화·갱년기 여성의 건강에 도움을 줄 수 있음,"의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의"
1,①피부건강·항산화·면역력 증진·혈중 콜레스테롤 개선에 도움을 줄 수 있음,NaN
2,혈행 개선에 도움을 줄 수 있음(기타기능II) ·혈중 중성지질 개선에 도움을 줄 수 있음 (기타기능II),NaN
3,①항산화·높은 혈압 감소에 도움을 줄 수 있음,NaN
4,①관절 및 연골 건강·피부보습에 도움을 줄 수 있음,게 또는 새우에 알레르기가 있는 사람은 섭취에 주의(게 또는 새우를 원재료로 사용한 경우에 한함)
...,...,...
154,(국문) ① 피부 보습에 도움을 줄 수 있음\n ② 자외선에 의한 피부 손상으로부터 피부 건강을 유지하는데 도움을 줄 수 있음\n (영문) ① May help to moisturize (dry) skin\n ② May help to maintain good skin health by protecting against damage from UV exposure,"(1) 영‧유아, 어린이, 임산부 및 수유부는 섭취를 피할 것\n(2) 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의할 것\n(3) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
155,(국문) ① 피부 보습에 도움을 줄 수 있음\n ② 자외선에 의한 피부 손상으로부터 피부 건강을 유지하는데 도움을 줄 수 있음\n(영문) ① May help to maintain skin moisturizing\n ② May help to maintain skin health from skin damage by UV radiation,"(1) 영‧유아, 어린이, 임산부 및 수유부는 섭취를 피할 것\n(2) 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의할 것\n(3) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
156,(국문) 위 점막 내 헬리코박터균(Helicobacter pylori) 증식을 억제하고 위 점막을 보호하여 위 건강에 도움을 줄 수 있음\n(영문) May help to stomach health by protecting gastric mucosa and inhibiting proliferation of Helicobacter pylori in gastric mucosa.,"-영・유아, 어린이, 임산부 및 수유부는 섭취에 주의\n-특정 질환(알레르기 체질 등)이 있는 분은 섭취에 주의\n-이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
157,혈행 개선에 도움을 줄 수 있음(생리활성기능 2등급),-대두에 알레르기를 나타내는 사람은 섭취에 주의\n-임산부․수유부 섭취에 주의\n-의약품(혈액항응고제) 복용 시 섭취에 주의\n-수술 전․후 섭취에 주의


- 단어 사이의 연결자로 사용되는 기호가 다양하게 사용되고 있어 `'・'`, `'‧'`, `'·'`, `'･'`, '․'의 경우 제거하지 않고 `'·'`로 통합

In [360]:
find_chars_df(i_df, ['ingredient_function', 'ingredient_caution'], ['`'])[['ingredient_function', 'ingredient_caution']]

ingredient_caution 컬럼값에 "`" 문자열이 없습니다.


,ingredient_function,ingredient_caution
0,(국문)\n- 갱년기 여성의 건강에 도움을 줄 수 있음\n- 피부보습에 도움을 줄 수 있음\n- 자외선에 의한 피부 손상으로부터 피부건강을 유지하는데 도움을 줄 수 있음\n(영문)\n- May help to menopausal women`s health\n- May help to maintain skin moisturizing\n- May help to maintain skin health from skindamage by UV radiation,"(1) 어린이, 임산부와 수유부는 섭취를 피하는 것이 좋습니다. (2) 에스트로겐 호르몬에 민감한 사람은 섭취에 주의하시기 바랍니다."


- '`' 기호의 경우 영문 번역본에 포함된 내용이기 때문에 함께 제거될 기호이다.

In [361]:
find_chars_df(i_df, ['ingredient_function', 'ingredient_caution'], ['β'])[['ingredient_function', 'ingredient_caution']]

ingredient_function 컬럼값에 "β" 문자열이 없습니다.


,ingredient_function,ingredient_caution
0,①혈중 콜레스테롤 개선에 도움을 줄 수 있음,베타-카로틴(β-carotene)의 흡수를 저해할 수 있음
1,①눈의 피로도 개선에 도움을 줄 수 있음,과다 섭취 시 일시적으로 피부가 황색으로 변할 수 있음\nβ-카로틴의 흡수를 저해할 수 있음
2,(국문) 관절건강에 도움을 줄 수 있음\n(영문) May help to maintain healthy joint,"(1) 영·유아, 어린이, 임산부 및 수유부는 섭취에 주의\n(2) 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의\n(3) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n(4) 질환이 있거나 의약품 복용 시 전문가와 상담할 것\n(5) 과다 섭취 시 일시적으로 피부가 황색으로 변할 수 있음\n(6) β-카로틴의 흡수를 저해할 수 있음"


- 'β'는 베타카로틴 관련 기호로 사용되기 때문에 제거하지 않는다.

In [362]:
find_chars_df(i_df, ['ingredient_function', 'ingredient_caution'], ['‘', '’'])[['ingredient_function', 'ingredient_caution']]

ingredient_function 컬럼값에 "‘" 문자열이 없습니다.


,ingredient_function,ingredient_caution
0,(국문)\n-인체에 유해한 활성산소를 제거하는데 도움을 줄 수 있음\n-혈액의 흐름을 방해할 수 있는 혈소판응집을 억제하는데 도움을 줄 수 있음\n-갱년기 여성의 건강에 도움을 줄 수 있음\n(영문)\n-May help to remove reactive oxygen species which are harmful to human body\n-May help to inhibit blood coagulatin of platelet which can disturb blood flow\n-May help to climacteric women’s health,"-임산부 및 수유부와 12세 이하의 어린이는 섭취를 피할 것\n-수술 전후, 질환이 있거나 의약품 복용 시 전문가와 상담할 것\n-식사 후 또는 식사와 함께 섭취할 것\n-이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
1,햇볕 또는 자외선에 의한 피부손상으로부터 피부 건강을 유지하는데 도움을 줄 수 있음(기타기능II),"-임산부, 수유부 및 어린이는 섭취를 피해야 함\n-‘달맞이꽃종자유’에 과민증이 있는 사람을 섭취를 삼가야 함\n-과량 섭취 시 위장장애 등의 부작용이 있을 수 있음\n-기타 질병을 보유하고 있는 사람 및 수술 전후 환자는 섭취를 삼가야 하며 섭취 시 의사와 상담해야 함"


- 따옴표 기호는 상세 내용을 나타내는 기호로 사용되기 때문에 제거하지 않는다.

In [363]:
find_chars_df(i_df, ['ingredient_function', 'ingredient_caution'], ['※'])[['ingredient_function', 'ingredient_caution']]

ingredient_function 컬럼값에 "※" 문자열이 없습니다.


,ingredient_function,ingredient_caution
0,(국문) 면역 기능에 도움을 줄 수 있음\n(영문) May help immune function,"(1) 질환이 있거나 의약품 복용 시 전문가와 상담할 것\n(2) 특정질환(알레르기 체질, 단장증후군, 유당불내증 등)이 있는 분은 섭취에 주의할 것\n(3) 영·유아, 어린이, 임산부 및 수유부는 섭취에 주의할 것\n(4) 어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도할 것\n(5) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n※ 유산균의 정성적 품질관리를 위해 원 균주(전장유전체 정보 등)를 주기적으로 관리하여야 하며, 식약처 요청 시 관련자료를 제출하여야 함"


- '※' 기호는 제거해도 괜찮은 것으로 확인

In [364]:
find_chars_df(i_df, ['ingredient_function', 'ingredient_caution'], ['①', '②', '③', '④', '⑤'])[['ingredient_function', 'ingredient_caution']]

ingredient_caution 컬럼값에 "④" 문자열이 없습니다.
ingredient_caution 컬럼값에 "⑤" 문자열이 없습니다.


,ingredient_function,ingredient_caution
0,①정상적인 면역기능에 필요②정상적인 세포분열에 필요,NaN
1,①체내 산소운반과 혈액생성에 필요②에너지 생성에 필요,특히 6세 이하는 과량섭취하지 않도록 주의
2,①단백질 및 아미노산 이용에 필요②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요,NaN
3,①에너지 이용에 필요②신경과 근육 기능 유지에 필요,NaN
4,①갑상선 호르몬의 합성에 필요②에너지 생성에 필요③신경발달에 필요,NaN
...,...,...
73,장내유익균의 증식과 유해균의 억제에 도움을 줄 수 있음(기타기능II)배변활동을 원활히 하는데 도움을 줄 수 있음(기타기능II),①과량으로 섭취 시에는 설사를 유발할 수 있습니다.
74,건강한 혈당유지에 도움,"①기관지 천식, 해소 기침이나 기도에 심한 염증이 있는 사람은 사용을 금하시기 바랍니다."
75,혈압이 걱정되시는 분에게 도움을 줄 수 있습니다.(기타기능II),"① 임산부, 수유부는 섭취를 삼가시기 바랍니다.② 어린이, 노인, 신부전이 있는 사람, 혈압약을 복용하시는 분이 섭취 시에는 의사와 상담하셔야 합니다."
76,노인의 인지능력저하 개선에 도움을 줄 수 있음(기타기능II),"①소화불량, 속쓰림 등이 나타날 수 있습니다.②혈액응고방지제 또는 혈당강하제를 복용하시는 분은 의사의 상담 하에 사용하여 주십시오."


In [365]:
find_chars_df(i_df, ['ingredient_function', 'ingredient_caution'], ['ⓛ'])[['ingredient_function', 'ingredient_caution']]

ingredient_caution 컬럼값에 "ⓛ" 문자열이 없습니다.


,ingredient_function,ingredient_caution
0,ⓛ 식후 혈당상승 억제에 도움을 줄 수 있음,NaN


- 숫자 관련 특수문자 기호도 구분자로 사용되었기 떄무에 제거 후 구분자로 건강기능 구분을 한다.

In [366]:
find_chars_df(i_df, ['ingredient_function', 'ingredient_caution'], ['다.', '음.', '오.'])[['ingredient_function', 'ingredient_caution']]

ingredient_function 컬럼값에 "오." 문자열이 없습니다.


,ingredient_function,ingredient_caution
0,혈압이 높은 사람에게 도움을 줄 수 있습니다. (기타기능Ⅱ),"① 임산부와 수유기 여성, 어린이는 섭취에 주의하시기 바람"
1,인체의 항산화능 증진에 도움을 줄 수 있습니다.(기타기능 II),"① 위장장애가 발생할 수 있다.② 영유아, 임산부는 섭취를 삼가 해야 한다.③ 식사조절, 운동을 병행하는 것이 체지방 감소에 효과적이다."
2,혈압이 걱정되시는 분에게 도움을 줄 수 있습니다.(기타기능II),"① 임산부, 수유부는 섭취를 삼가시기 바랍니다.② 어린이, 노인, 신부전이 있는 사람, 혈압약을 복용하시는 분이 섭취 시에는 의사와 상담하셔야 합니다."
3,간건강에 도움을 줄 수 있습니다.,NaN
4,"장내 유익균의 증식과 유해균의 억제에 도움을 줄 수 있습니다, 배변활동을 원활히 하는데 도움을 줄 수 있습니다.",① 과량으로 섭취 시에는 설사를 유발할 수 있습니다.
5,(국문) 콩발효추출물은 발효시킨 콩을 추출한 것으로 당의 흡수를 억제하여 식후혈당을 건강하게 유지하는데 도움을 줄 수 있는 식품입니다.\n(영문) May help to maintain healthy blood glucose level,NaN
6,혈압이 높은 사람에게 도움을 줄 수 있습니다.(기타기능 II),NaN
7,혈중 콜레스테롤 개선에 도움을 줄 수 있습니다.(기타기능 II),"-카페인이 함유되어 있어 초조감, 불면 등을 나타낼 수 있음\n-임산부, 수유 여성, 어린이, 알레르기성 질환보유자, 질병치료 중인 환자는 섭취에 주의"
8,-산화스트레스로부터 인체를 보호하는데 도움을 줄 수 있습니다.\n-혈관벽 두께(내중막 두께 : IMT) 증가 억제를 통한 혈행개선에 도움을 줄 수 있음\n-자외선에 의한 피부홍반개선으로 피부건강에 도움을 줄 수 있음,밀 단백질에 알레르기를 나타내는 사람은 섭취에 주의
9,"유익균 증식, 유해균 억제에 도움을 줄 수 있습니다.(생리활성기능 2등급))",과량 섭취 시 위장관계 계통의 이상반응 및 전해질 이상 등이 나타날 수 있음


In [367]:
find_chars_df(i_df, ['ingredient_function', 'ingredient_caution'], ['임산부.'])[['ingredient_function', 'ingredient_caution']]

ingredient_function 컬럼값에 "임산부." 문자열이 없습니다.


,ingredient_function,ingredient_caution
0,식후혈당 상승 억제에 도움을 줄 수 있음(생리활성기능 2등급),-임산부.수유부는 섭취에 주의


- `'.'` 의 경우 구분자로 사용하는 경우와 단어 사이에 연결자 역할을 하는 경우가 있음
- `'다.'`, `'음.'`, `'오.'` 로 `.`이 오는 경우 구분자 =>  `'\n'`
- `'임산부.수유부'` 의 경우와 같을 때는 `', '`로 대체

### 7-2. 건강기능 및 주의사항 컬럼 텍스트 전처리
- 영양성분의 건강기능과 섭취시 주의사항에 대한 TEXT 가 담긴 `ingredient_function`, `ingredient_caution` 컬럼을 정규식을 이용해 전처리
- [정규식 해석 사이트](https://regexr.com/) 참고

In [368]:
# 정규식을 이용해 텍스트 정제하여 데이터프레임 생성
def cleaned_df_data(df, col_name, cleaned_col_name):
    # null값일 경우 정규식 전처리가 불가능하기 때문에 null값이 없는 행만 전처리 진행
    data = df[df[col_name].notnull()][col_name]
    
    # 전처리하는 행의 인덱스를 리스트로 저장
    index_list = data.index.to_list()
    
    # 정제된 텍스트를 넣어줄 빈 리스트 생성
    cleaned_text_list = []
    
    # 데이터프레임의 컬럼값들을 순회하기 위해 for 구문 사용
    for cleaned_text in data:
        
        # '(국문)' 또는 '[국문]' 또는 '(영문)' 또는 '[영문]'이라는 문자 제거
        if '(국문)' in cleaned_text or '(영문)' in cleaned_text or '[국문]' in cleaned_text or '[영문]' in cleaned_text:
            # '(영문)' 또는 '[영문]'과 해당 문자열 뒤에 모든 문자열 제거
            cleaned_text = re.sub(r'(\(영문\)|\[영문\])[\s\S]*', '', cleaned_text)
            # '(국문)' 또는'[국문]'이라는 문자 제거
            cleaned_text = re.sub(r'\(국문\)|\[국문\]', '', cleaned_text)
            
        # '(기타기능II)' 또는 '(기타II 등급)' 또는 '(기타 II)' 또는 '(기타Ⅱ)' 또는 '(생리활성기능)' 또는'(생리활성기능2등급)'이라는 문자가 있을 경우 띄어쓰기로 대체
        if '(기타' in cleaned_text or '(생리활성기능' in cleaned_text:
            cleaned_text = re.sub(r'\(기타[^)]*\)|\(생리활성기능[^)]*\)', ' ', cleaned_text)
        
        # '(가)' or '[가]' 와 같이 소괄호와 대괄호 안에 '가', '나', '다'... 만 있을 경우 제거
        # 괄호 안에 공백이 같이 있을 경우 함께 제거 필요
        cleaned_text = re.sub(r'\[[가나다라마바사아자차카타파하\s]]*\]|\([가나다라마바사아자차카타파하\s]*\)', '', cleaned_text)
        
        # '(1)' 소괄호와 대괄호 안에 숫자만 있을 경우 => '\n'으로 대체
        # 괄호 안에 숫자와 공백이 같이 있을 경우 함께 처리 필요
        cleaned_text = re.sub(r'[\n]*(\[[0-9\s]*\])|[\n]*(\([0-9\s]*\))', '\n', cleaned_text)
        
        # '1)', '2)'와 같은 문자열 => '\n'으로 대체
        cleaned_text = re.sub(r'([\n]*[0-9]+\))', '\n', cleaned_text)
        
        # '1.' 같이 숫자 뒤에 '.'이 있을 경우 => '\n'으로 대체
        cleaned_text = re.sub(r'[\n]*[0-9]+[.]', '\n', cleaned_text)
        
        # '‘', '’ '일 경우 ''으로 대체하여 제거
        cleaned_text = re.sub(r'[‘’]+', '', cleaned_text)        
        
        # '+R-45' 예외 상황 처리 => ''으로 대체하여 제거
        cleaned_text = re.sub(r'\+R-45', '', cleaned_text) 
        
        # '섭취 시 주의사항 : ' 예외 상황 처리 => ''으로 대체하여 제거
        cleaned_text = re.sub(r'섭취 시 주의사항 : ', '', cleaned_text) 
        
        # '임산부.수유부' 예외 사항 처리 => '.' 기호를 ','로 대체
        cleaned_text = re.sub(r'임산부\.수유부', '임산부, 수유부', cleaned_text)
        
        # '․', '・', '‧', '·', '‧' '･' 특수문자 하나로 통일 => '·'
        cleaned_text = re.sub(r'[\․\·\‧\･\‧\・]+', '·', cleaned_text)
        
        # 특수문자 제거
        # 1. 소괄호 안에 특수 문자가 있는 경우, 소괄호와 그 안의 특수문자를 제외
        # 2. '-', '.', ',', ':', '·', 'β' 를 제외한 모든 특수문자 => '\n'로 대체
        cleaned_text = re.sub(r'(?!\([^)]+\))[^a-zA-Z0-9가-힣\s\)\-\.\,\:\·β]+', '\n', cleaned_text)

        # '다', '음', '오' 다음에 ',' 이 오는 경우 ',' => ',' 를 '\n'로 대체
        cleaned_text = re.sub(r'다\,+[\n]*', '다\n', cleaned_text)
        cleaned_text = re.sub(r'음\,+[\n]*', '음\n', cleaned_text)
        cleaned_text = re.sub(r'오\,+[\n]*', '오\n', cleaned_text)
    
        # white space가 2번이상 나올 경우 ' ' 로 대체      
        cleaned_text = re.sub(r'[\s]{2,}', ' ', cleaned_text)
        # linebreak 이후 '-' 기호 '\n' 로 대체
        cleaned_text = re.sub(r'[\n]+[\s\-\)]+', '\n', cleaned_text)
        # white space 뒤에 '-' 기호가 나올경우 ' '띄어쓰기로 대체
        cleaned_text = re.sub(r'[\s]+[\-]', ' ', cleaned_text)
        
        # 양쪽 공백문자 제거, 탭 제거
        cleaned_text = cleaned_text.strip()   
        
        # # '\n'기준으로 split
        # cleaned_text = cleaned_text.split('\n')
        
        # '·', '-', '+', whitespace로 시작할 경우 제거
        cleaned_text = re.sub(r"^[\·\-\+\s]+", "", cleaned_text)
        # cleaned_text = [re.sub(r"^[\·\-\+\s]+", "", x) for x in cleaned_text]
            
        # 텍스트 정제후 데이터를 리스트에 추가
        cleaned_text_list.append(cleaned_text)
    
    # 기존 데이터프레임에 정제된 데이터가 담길 수 있게 컬럼 생성 후 값 추가
    df[cleaned_col_name] = np.nan
    
    for idx in range(len(index_list)):
        # 데이터프레임의 열에 리스트를 할당
        df.at[index_list[idx], cleaned_col_name] = cleaned_text_list[idx]
        
    return df, cleaned_text_list

In [369]:
# 전처리 함수 적용
i_df, cleaned_function_data = cleaned_df_data(i_df, 'ingredient_function', 'ingredient_function_content')
i_df, cleaned_caution_data = cleaned_df_data(i_df, 'ingredient_caution', 'ingredient_caution_content')

# 확인
i_df[['ingredient_function', 'ingredient_function_content', 'ingredient_caution', 'ingredient_caution_content']][10:20]

,ingredient_function,ingredient_function_content,ingredient_caution,ingredient_caution_content
10,①혈중 콜레스테롤 개선에 도움을 줄 수 있음,혈중 콜레스테롤 개선에 도움을 줄 수 있음,대두단백에 알레르기를 나타내는 경우에는 섭취 주의,대두단백에 알레르기를 나타내는 경우에는 섭취 주의
11,①피부건강·항산화·면역력 증진·혈중 콜레스테롤 개선에 도움을 줄 수 있음,피부건강·항산화·면역력 증진·혈중 콜레스테롤 개선에 도움을 줄 수 있음,NaN,NaN
12,①혈중 콜레스테롤 개선에 도움을 줄 수 있음,혈중 콜레스테롤 개선에 도움을 줄 수 있음,베타-카로틴(β-carotene)의 흡수를 저해할 수 있음,베타-카로틴(β-carotene)의 흡수를 저해할 수 있음
13,①면역기능 개선에 도움을 줄 수 있음,면역기능 개선에 도움을 줄 수 있음,NaN,NaN
14,"장내 유익균 증식, 유해균 억제, 배변활동에 도움을 줄 수 있음 (기타기능 II)","장내 유익균 증식, 유해균 억제, 배변활동에 도움을 줄 수 있음","① 어린이, 임산부, 수유기 여성은 섭취에 주의하십시오.② 과량 섭취 시 복부팽만감을 느낄 수 있습니다.","어린이, 임산부, 수유기 여성은 섭취에 주의하십시오. 과량 섭취 시 복부팽만감을 느낄 수 있습니다."
15,높은 혈중 콜레스테롤 수치개선에 도움,높은 혈중 콜레스테롤 수치개선에 도움,①식물 스타놀 에스테르는 지용성 비타민의 흡수를 저해할 수 있습니다.,식물 스타놀 에스테르는 지용성 비타민의 흡수를 저해할 수 있습니다.
16,갱년기 여성의 건강에 도움을 줄 수 있음(기타Ⅱ),갱년기 여성의 건강에 도움을 줄 수 있음,① 임산부와 수유부는 섭취를 피하는 것이 좋습니다.② 항혈전제를 복용하시는 분은 의사와 상의하시기 바랍니다.③ 에스트로겐 호르몬에 민감한 사람은 섭취에 주의하시기 바랍니다.,임산부와 수유부는 섭취를 피하는 것이 좋습니다. 항혈전제를 복용하시는 분은 의사와 상의하시기 바랍니다. 에스트로겐 호르몬에 민감한 사람은 섭취에 주의하시기 바랍니다.
17,혈압이 높은 사람에게 도움을 줄 수 있습니다. (기타기능Ⅱ),혈압이 높은 사람에게 도움을 줄 수 있습니다.,"① 임산부와 수유기 여성, 어린이는 섭취에 주의하시기 바람","임산부와 수유기 여성, 어린이는 섭취에 주의하시기 바람"
18,이 제품은 중쇄지방산을 함유하고 있어 다른 식용유와 비교하였을 때 체지방 증가가 적을 수 있음(기타기능II),이 제품은 중쇄지방산을 함유하고 있어 다른 식용유와 비교하였을 때 체지방 증가가 적을 수 있음,NaN,NaN
19,혈압이 높은 사람에게 도움을 줄 수 있습니다(기타기능 II),혈압이 높은 사람에게 도움을 줄 수 있습니다,NaN,NaN


### 7-3. 한국어 형태소 분석기로 명사 단어 분류
- 건강기능과 주의사항 컬럼을 코드(카테고리)별로 분류할 수 있게 한국어 형태소 분석기로 명사 단어로 분류 작업

In [370]:
# !pip install konlpy
# !pip install nltk
# !pip install PyKomoran
# !pip install ydata-profiling # !conda install -c conda-forge ydata-profiling

In [371]:
# cleaned_function_data2 = [x.split('\n') for x in cleaned_function_data]

# # 리스트 평탄화
# flatten_function_list = [item for sublist in cleaned_function_data2 for item in sublist]

# flatten_function_list

In [372]:
# 한국어 형태소 분석기 사용할 단어 형태로 처리해 데이터프레임을 반환하는 함수 정의
def create_text_to_word(text_list):
    # 텍스트 리스트를 '\n' 구분자로 분리하고 평탄화
    flatten_list = [text for sublist in text_list for text in sublist.split('\n')]
    
    # 텍스트를 하나의 문자열로 합치고 특수문자를 띄어쓰기(' ')로 대체
    cleaned_text = re.sub(r'[^가-힣A-Za-z0-9\s]', ' ', ', '.join(flatten_list))
    
    # 공백을 기준으로 split 후 중복 제거 및 정렬
    cleaned_text_list = sorted(set(cleaned_text.split()))
    
    # 데이터프레임 생성
    cleaned_word_df = pd.DataFrame({'word': cleaned_text_list})
    
    return cleaned_word_df

In [373]:
# 건강기능 text 데이터 함수 적용
function_word_df = create_text_to_word(cleaned_function_data)
# 주의사항 text 데이터 함수 적용
caution_word_df = create_text_to_word(cleaned_caution_data)

# 확인
print(function_word_df[20:30])
print(caution_word_df[20:30])

     word
20    갱년기
21  걱정되시는
22     건강
23    건강에
24    건강을
25    건강의
26   건강하게
27    건강한
28    건조한
29    것으로
   word
20   강황
21  개선에
22  개인에
23  건강에
24    것
25   것을
26   것이
27    게
28   결핵
29  경미한


In [374]:
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from PyKomoran import Komoran, DEFAULT_MODEL

- Windows 에서 지원하지 않는 Mecab을 제외한 Kkma, Komoran, Hannanum, Okt 4가지 형태소 분석기를 비교

In [375]:
# 명사 추출 함수 정의
def extract_nouns(analyzer, word):
    tokens = ' '.join(analyzer.nouns(word))
    return tokens if tokens else word # 분석기에서 명사로 처리되지 않는 것은 그 단어 그대로 나오게 한다.

# 분석할 단어를 형태소 분석기의 명사 처리하는 함수 정의
def create_konlpy_nonus(df, word_col):
    global kkma, komoran, hannanum, okt
    
    df['kkma'] = df[word_col].apply(lambda word: extract_nouns(kkma, word))
    df['komoran'] = df[word_col].apply(lambda word: extract_nouns(komoran, word))
    df['hannanum'] = df[word_col].apply(lambda word: extract_nouns(hannanum, word))
    df['okt'] = df[word_col].apply(lambda word: extract_nouns(okt, word))
    
    return df

In [376]:
# 건강기능 함수 적용 확인
kkma=Kkma()
komoran = Komoran(DEFAULT_MODEL['FULL'])
hannanum = Hannanum()
okt=Okt()

function_word_df = create_konlpy_nonus(function_word_df, 'word')
function_word_df

,word,kkma,komoran,hannanum,okt
0,4,4,4,4,4
1,Diacylglyceride,Diacylglyceride,Diacylglyceride,Diacylglyceride,Diacylglyceride
2,Helicobacter,Helicobacter,Helicobacter,Helicobacter,Helicobacter
3,IMT,IMT,IMT,IMT,IMT
4,PMO,PMO,PMO,PMO,PMO
...,...,...,...,...,...
370,흐름을,흐름,흐름,흐름,흐름
371,흡수되고,흡수,흡수,흡수,흡수
372,흡수를,흡수,흡수,흡수,흡수
373,흡수에,흡수,흡수,흡수,흡수


In [377]:
# 주의사항 함수 적용 확인
caution_word_df = create_konlpy_nonus(caution_word_df, 'word')
caution_word_df

,word,kkma,komoran,hannanum,okt
0,12세,12 12세 세,12세,12세,세
1,6개월,6 6개월 개월,6개월,6개월,개월
2,6세,6 6세 세,6세,6세,세
3,8주,8 8주 주,8주,8주,주
4,AIDS,AIDS,AIDS,AIDS,AIDS
...,...,...,...,...,...
602,후,후,후,후,후
603,후에,후,후,후,후
604,후에는,후,후,후,후
605,흡수를,흡수,흡수,흡수,흡수


In [378]:
caution_word_df

,word,kkma,komoran,hannanum,okt
0,12세,12 12세 세,12세,12세,세
1,6개월,6 6개월 개월,6개월,6개월,개월
2,6세,6 6세 세,6세,6세,세
3,8주,8 8주 주,8주,8주,주
4,AIDS,AIDS,AIDS,AIDS,AIDS
...,...,...,...,...,...
602,후,후,후,후,후
603,후에,후,후,후,후
604,후에는,후,후,후,후
605,흡수를,흡수,흡수,흡수,흡수


- 건강기능 및 주의사항에 사용되는 용어를 가장 잘 처리하는 Hannanum을 사용하는 것이 적절한 것으로 보인다.

- 한국어 형태소 분석기 별 소요시간 확인
- [함수 REF SITE](https://blog.cosadama.com/articles/2021-practicenlp-01/)

In [379]:
import time

def  time_use(df, word_col, a, b):
    start = time.time()
    for i in  range(0, 372): #문서의 형태, 크기에 따라서 다르게 설정하기
        contents = df[word_col].loc[i] #말뭉치 설정
        nouns = a.nouns(contents)
        globals()['Var_{0}{1}'.format(b, i)]=nouns
    end = time.time()
    total_time=(f"{end - start:.5f} sec")
    print('{0} ==> {1}'.format(a ,total_time))

In [380]:
print('<건강기능 단어 별 형태소 분석기 명사 단어 처리 소요시간>')
time_use(function_word_df, 'word', kkma, 'kkma')
time_use(function_word_df, 'word', komoran, 'komoran')
time_use(function_word_df, 'word', hannanum, 'hannanum')
time_use(function_word_df, 'word', okt,'okt')

print('\n<주의사항 단어 별 형태소 분석기 명사 단어 처리 소요시간>')
time_use(caution_word_df, 'word', kkma, 'kkma')
time_use(caution_word_df, 'word', komoran, 'komoran')
time_use(caution_word_df, 'word', hannanum, 'hannanum')
time_use(caution_word_df, 'word', okt,'okt')

<건강기능 단어 별 형태소 분석기 명사 단어 처리 소요시간>
<konlpy.tag._kkma.Kkma object at 0x000001DBAC056460> ==> 0.72406 sec
<PyKomoran.core.Komoran object at 0x000001DB829B3AF0> ==> 0.23625 sec
<konlpy.tag._hannanum.Hannanum object at 0x000001DB829A8A30> ==> 0.05789 sec
<konlpy.tag._okt.Okt object at 0x000001DB829A89A0> ==> 0.06678 sec

<주의사항 단어 별 형태소 분석기 명사 단어 처리 소요시간>
<konlpy.tag._kkma.Kkma object at 0x000001DBAC056460> ==> 0.63938 sec
<PyKomoran.core.Komoran object at 0x000001DB829B3AF0> ==> 0.23496 sec
<konlpy.tag._hannanum.Hannanum object at 0x000001DB829A8A30> ==> 0.05584 sec
<konlpy.tag._okt.Okt object at 0x000001DB829A89A0> ==> 0.06230 sec


- 단어 기준 처리속도는 Okt > Hannanum > Komoran > Kkma 순으로 빠르다
- 해당 전처리는 서비스에서 실시간 서비스로 사용되는 것은 아니기 때문에 속도가 크게 중요하지는 않지만, 필요한 명사 분류 및 속도가 비교적 빠른 편인 Hannanum으로 사용하기로 결정

### 7-4. 불용어 정리
- `function_word_df` 와 `caution_word_df`의 'hannanum' 컬럼에서 제거할 불용어 확인

In [381]:
check_word_df = pd.concat([function_word_df[['word', 'hannanum']], caution_word_df[['word', 'hannanum']]], axis=0)
check_word_df.drop_duplicates(subset=['word'], inplace=True, ignore_index=True) # 중복된 단어 979 -> 656개

In [382]:
caution_word_df.drop_duplicates(subset=['hannanum'], inplace=True, ignore_index=True)

In [383]:
check_word_df[900:920]

,word,hannanum
900,항히스타민제,항히스타민제
901,해로울,해로울
902,해소,해소
903,해야,해야
904,해조류,해조류
905,해조식,해조식
906,혈당강하제,혈당강하제
907,혈당강하제를,혈당강하제
908,혈압강하제,혈압강하제
909,혈압약,혈압약


In [384]:
check_keyword = '확인하신'
function_caution_mask_a = (i_df['ingredient_function_content'].notnull()) & (i_df['ingredient_function_content'].str.contains(check_keyword))
function_caution_mask_b = (i_df['ingredient_caution_content'].notnull()) & (i_df['ingredient_caution_content'].str.contains(check_keyword))
i_df[(function_caution_mask_a) | (function_caution_mask_b)][['ingredient_function_content', 'ingredient_caution_content']]

,ingredient_function_content,ingredient_caution_content
136,"혈액의 흐름을 방해할 수 있는 혈소판응집을 억제하여, 혈액흐름을 원활히 하는데 도움을 줄 수 있음.","알레르기 체질이신 분은 원재료를 확인하신 후 섭취하십시오\n임산부와 수유기 여성은 섭취를 피하는 것이 좋습니다\n과다 섭취 시 혈액응고가 저해되어 출혈 위험이 증가될 수 있습니다\n항혈전 관련 약품(항응고제, 항혈소판제제)을 섭취하는 사람은 의사와 상담 후 섭취하여야 합니다\n수술 전과 후에는 섭취를 피해야 합니다"


In [385]:
# 불용어 -> 모듈화해서 stopwords.py 의 stopwords에 리스트 정리
# 잘못 분류되는 명사 -> updated_nouns_words에 정리 -> 원래 word 그대로 나오게 한다.
from stopwords import stopwords
from updated_nouns_words import updated_nouns_words
print(f'stopwords: {stopwords}')
print(f'updated_nouns_words: {updated_nouns_words}')

stopwords: ['가', '것', '곳', '과', '대하여', '대한', '대해', '도움', '등', '따라', '때', '또는', '를', '마', '및', '바', '반드시', '본', '수', '시', '와의', '위해', '유지', '은', '을', '의', '의한', '의해', '이', '인해', '있거나', '있다', '있다는', '있으니', '있으며', '있으므로', '있습니다', '있어', '있으신', '있을', '있음', '자', '정', '좋습니다', '좋음', '주', '줄', '줌', '통한', '통해', '특히', '필요', '하', '하고', '하는', '하는데', '하며', '하여', '한', '한다', '한함', '할', '함', '합니다', '해야']
updated_nouns_words: ['', '계획하', '낮', '은', '스테로이드계', '속쓰', '알러', '영유', '와파', '밀알부민', '것', '호모시스테']


In [386]:
# Hannanum으로 명사 분석하는 함수 정의
def process_with_hannanum_for_stopwords(words, stopwords, updated_nouns_words):
    nonus_words = []
    for word in words:
        # nonus = extract_nouns(hannanum, word)
        nonus = hannanum.nouns(word)
        nonus_word = ' '.join(nonus)
        # 해당 리스트에 없을 경우 문자열로 변환 / 있을 경우에는 원래 문자열 그대로 word로 지정
        if nonus_word in updated_nouns_words: 
            nonus_word = word
            
        if nonus_word not in stopwords: # 불용어 제거
            nonus_words.append(nonus_word)
            
    nonus_words = ' '.join(nonus_words) # 문자열로 변환
    return nonus_words

# 불용어를 확인하기 위한 text 전처리와 Hannanum 명사 분석하는 함수정의
def process_text_for_stopwords(text_list, stopwords, updated_nouns_words):
    # 텍스트 리스트를 평탄화하여 하나의 리스트로 변환
    flatten_list = [text for sublist in text_list for text in sublist.split('\n')]
    
    # 특수문자를 ' '로 대체하고 단어별로 split
    cleaned_data = [re.sub(r'[^가-힣A-Za-z0-9]', ' ', x) for x in flatten_list]
    cleaned_data = [x.split() for x in cleaned_data]
    
    # 중복 제거
    check_text_df = pd.DataFrame({'origin_text': flatten_list, 'word_text': cleaned_data})
    check_text_df.drop_duplicates(subset=['origin_text'], inplace=True, ignore_index=True)
    
    # Hannanum 명사 분석
    check_text_df['hannanum'] = check_text_df['word_text'].apply(lambda words: process_with_hannanum_for_stopwords(words, stopwords, updated_nouns_words))
    
    return check_text_df

In [387]:
function_text_df = process_text_for_stopwords(cleaned_function_data, stopwords, updated_nouns_words)
caution_text_df = process_text_for_stopwords(cleaned_caution_data, stopwords, updated_nouns_words)

In [388]:
function_text_df.head()

,origin_text,word_text,hannanum
0,정상적인 면역기능에 필요,"[정상적인, 면역기능에, 필요]",정상적 면역기능
1,정상적인 세포분열에 필요,"[정상적인, 세포분열에, 필요]",정상적 세포분열
2,체내 산소운반과 혈액생성에 필요,"[체내, 산소운반과, 혈액생성에, 필요]",체내 산소운반 혈액생성
3,에너지 생성에 필요,"[에너지, 생성에, 필요]",에너지 생성
4,단백질 및 아미노산 이용에 필요,"[단백질, 및, 아미노산, 이용에, 필요]",단백질 아미노산 이용


In [389]:
caution_text_df[['origin_text', 'hannanum']][:20]

,origin_text,hannanum
0,특히 6세 이하는 과량섭취하지 않도록 주의,6세 이하 과량섭취 않도록 주의
1,"의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의",의약품 당뇨치료제 혈액항응고제 복용 섭취 주의
2,대두단백에 알레르기를 나타내는 경우에는 섭취 주의,대두단백 알레르기 나타내는 경우 섭취 주의
3,베타-카로틴(β-carotene)의 흡수를 저해할 수 있음,베타 카로틴 carotene 흡수 저해
4,"어린이, 임산부, 수유기 여성은 섭취에 주의하십시오. 과량 섭취 시 복부팽만감을 느낄 수 있습니다.",어린이 임산부 수유기 여성 섭취 주의 과량 섭취 복부팽만감 느낄
5,식물 스타놀 에스테르는 지용성 비타민의 흡수를 저해할 수 있습니다.,식물 스타놀 에스테르 지용성 비타민 흡수 저해
6,임산부와 수유부는 섭취를 피하는 것이 좋습니다. 항혈전제를 복용하시는 분은 의사와 상의하시기 바랍니다. 에스트로겐 호르몬에 민감한 사람은 섭취에 주의하시기 바랍니다.,임산부 수유부 섭취 피하 것이 항혈전제 복용하시 분 의사 상의하시기 에스트로겐 호르몬 민감한 사람은 섭취 주의하시기
7,"임산부와 수유기 여성, 어린이는 섭취에 주의하시기 바람",임산부 수유기 여성 어린이 섭취 주의하시기 바람
8,"위장장애가 발생할 수 있다. 영유아, 임산부는 섭취를 삼가 해야 한다. 식사조절, 운동을 병행하는 것이 체지방 감소에 효과적이다.",위장장애 발생 영유아 임산부 섭취 삼가 식사조절 운동 병행 것이 체지방 감소 효과적
9,"어린이, 임산부 및 수유여성은 섭취에 주의",어린이 임산부 수유여성 섭취 주의


In [390]:
caution_text_df[caution_text_df['hannanum'].str.contains('수술')][['origin_text', 'hannanum']]

,origin_text,hannanum
34,"임산부, 수유부, 어린이 및 수술전후 환자는 섭취에 주의",임산부 수유부 어린이 수술전후 환자 섭취 주의
39,"임산부 및 수유부는 섭취를 피할 것 간·심장질환, 수술 전후, 고혈압, 당뇨 및 천식이 있거나 의약품(항응고제 등) 복용 시 전문가와 상담할 것 게 또는 새우에 알레르기가 있는 사람은 섭취에 주의할 것(게 또는 새우를 원재료로 사용한 경우) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것",임산부 수유부 섭취 피할 간 심장질환 수술 전후 고혈압 당뇨 천식 의약품 항응고제 복용 전문가 상담 게 새우 알레르기 있는 사람은 섭취 주의 게 새우 원재료 사용한 경우 이상사례 발생 섭취 중단 전문가 상담
40,"성인남성만 섭취할 것 수술 전후, 출혈성 질환이 있거나 항응고제 등 복용 시 전문가와 상담할 것 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음 메스꺼움 등 소화계통의 불편함과 설사를 유발할 수 있으니 식사 후 섭취할 것 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것",성인남성 섭취 수술 전후 출혈성 질환 항응고제 복용 전문가 상담 알레르기 체질 개인 과민반응 나타낼 메스꺼움 소화계통 불편함 설사 유발 식사 후 섭취 이상사례 발생 섭취 중단 전문가 상담
95,수술 전과 후에는 섭취를 피해야 합니다,수술 전과 후 섭취 피해
160,"수술 전후, 질환이 있거나 의약품 복용 시 전문가와 상담할 것",수술 전후 질환 의약품 복용 전문가 상담
181,기타 질병을 보유하고 있는 사람 및 수술 전후 환자는 섭취를 삼가야 하며 섭취 시 의사와 상담해야 함,기타 질병 보유 있는 사람 수술 전후 환자 섭취 삼가야 섭취 의사 상담해
206,수술 전·후 섭취에 주의,수술 전 후 섭취 주의
225,"영·유아, 어린이, 임산부 및 수유부는 섭취에 주의 수술 전·후 섭취에 주의 식사조절, 운동을 병행하는 것이 바람직함",영 유아 어린이 임산부 수유부 섭취 주의 수술 전 후 섭취 주의 식사조절 운동 병행 것이 바람직함
241,"임산부, 수유부, 청소년 및 어린이는 섭취를 피할 것 심혈관약 등 복용, 혈액응고장애가 있거나 수술예정인 사람은 전문가와 상담할 것 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것",임산부 수유부 청소년 어린이 섭취 피할 심혈관약 복용 혈액응고장애 수술예정인 사람은 전문가 상담 이상사례 발생 섭취 중단 전문가 상담
272,"영·유아, 어린이, 임산부 및 수유부는 섭취를 피할 것 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것 제안된 섭취량 이상 섭취 시 소화불량, 위장장애, 메스꺼움, 위산역류 등이 있을 수 있음 질환이 있거나 의약품 복용 시 전문가와 상담할 것 수술 전후에는 섭취에 주의",영 유아 어린이 임산부 수유부 섭취 피할 특정질환 알레르기 체질 있는 분 섭취 주의 이상사례 발생 섭취 중단 전문가 상담 제안 섭취량 이상 섭취 소화불량 위장장애 메스꺼움 위산역류 질환 의약품 복용 전문가 상담 수술 전후 섭취 주의


In [391]:
# 형태소 분석기로 `ingredient_function_content` 컬럼의 텍스트를 명사화하는 함수 정의
def created_text_to_nonus(df, text_col, text_nonus_col):
    global stopwords
    
    cleaned_function_datas = []
    
    for text in df[text_col]:
        text_lists = []
        if pd.notnull(text): # 결측치가 아닐 경우 - 리스트일 경우
            for line in text.split('\n'):
                # 특수문자 제거 및 단어별로 분리
                clean_words = [re.sub(r'[^가-힣A-Za-z0-9]', ' ', line).split()]

                # 형태소 분석을 통해 명사 추출 및 불용어 제거
                nonus_words = []
                for words in clean_words:
                    for word in words:
                        nonus_word = ' '.join(hannanum.nouns(word))
                        if nonus_word in updated_nouns_words: 
                            nonus_word = word
                        if nonus_word not in stopwords: # 불용어 제거
                            nonus_words.append(nonus_word)
                            
                text_lists.append(' '.join(nonus_words))
        else:
            text_lists = text
        cleaned_function_datas.append(text_lists)

    # 새로운 컬럼에 명사화된 TEXT 데이터 추가
    df[text_nonus_col] = cleaned_function_datas
    df[text_nonus_col] = df[text_nonus_col].apply(lambda x : re.sub(r'\,', '', ', '.join(x)) if isinstance(x, list) else np.nan)

    return df

In [392]:
# 함수적용
i_df = created_text_to_nonus(i_df, 'ingredient_function_content', 'function_nonus')
i_df = created_text_to_nonus(i_df, 'ingredient_caution_content', 'caution_nonus')

# 확인
i_df[['ingredient_function_content', 'function_nonus', 'ingredient_caution_content', 'caution_nonus']]

,ingredient_function_content,function_nonus,ingredient_caution_content,caution_nonus
0,정상적인 면역기능에 필요\n정상적인 세포분열에 필요,정상적 면역기능 정상적 세포분열,NaN,NaN
1,체내 산소운반과 혈액생성에 필요\n에너지 생성에 필요,체내 산소운반 혈액생성 에너지 생성,특히 6세 이하는 과량섭취하지 않도록 주의,6세 이하 과량섭취 않도록 주의
2,단백질 및 아미노산 이용에 필요\n혈액의 호모시스테인 수준을 정상으로 유지하는데 필요,단백질 아미노산 이용 혈액 호모시스테인 수준 정상,NaN,NaN
3,에너지 이용에 필요\n신경과 근육 기능 유지에 필요,에너지 이용 신경과 근육 기능,NaN,NaN
4,갑상선 호르몬의 합성에 필요\n에너지 생성에 필요\n신경발달에 필요,갑상선 호르몬 합성 에너지 생성 신경발달,NaN,NaN
...,...,...,...,...
431,위 점막을 보호하여 위 건강에 도움을 줄 수 있음,위 점막 보호 위 건강,임산부 및 수유부는 섭취를 피할 것 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의할 것 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것,임산부 수유부 섭취 피할 특정질환 알레르기 체질 있는 분 섭취 주의 이상사례 발생 섭취 중단 전문가 상담
432,근력개선에 도움을 줄 수 있음,근력개선,"영·유아, 어린이, 임산부 및 수유부는 섭취를 피할 것 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것 질환이 있거나 의약품 복용 시 섭취 전 전문가와 상담할 것",영 유아 어린이 임산부 수유부 섭취 피할 특정질환 알레르기 체질 있는 분 섭취 주의 이상사례 발생 섭취 중단 전문가 상담 질환 의약품 복용 섭취 전 전문가 상담
433,피부 보습에 도움을 줄 수 있음 자외선에 의한 피부손상으로부터 피부건강 유지에 도움을 줄 수 있음,피부 보습 자외선 피부손상 피부건강,"영·유아, 어린이, 임산부 및 수유부는 섭취를 피할 것 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의할 것 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것",영 유아 어린이 임산부 수유부 섭취 피할 특정질환 알레르기 체질 있는 분 섭취 주의 이상사례 발생 섭취 중단 전문가 상담
434,모발상태(윤기·탄력) 개선에 도움을 줄 수 있음,모발상태 윤기 탄력 개선,"영·유아, 어린이, 임산부 및 수유부는 섭취를 피할 것 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것",영 유아 어린이 임산부 수유부 섭취 피할 특정질환 알레르기 체질 있는 분 섭취 주의 이상사례 발생 섭취 중단 전문가 상담


## 8. 건강기능 코드 컬럼 생성
- `function_dict`를 순회하면서 데이터프레임의 token화된 `function_token` 컬럼의 문자열에서 `key`에 해당하는 건강기능 키워드 `value` 값이 있을 경우 
  - 건강기능 코드 컬럼에 값이 1, 없을 경우 0

In [393]:
def create_dict_col(df, col_name, code_dict):
    for key, values in code_dict.items():
        df[f'{key}'] = 0
        for value in values:
            df.loc[(df[col_name].notnull())& (df[col_name].str.contains(value)), [f'{key}']] = 1
    return df

In [394]:
from function_dict import function_dict
print(f'function_dict : {function_dict}')

i_df = create_dict_col(i_df, 'function_nonus', function_dict)
i_df[function_dict.keys()]

function_dict : {'HF00': ['아무것도 선택하지 않음'], 'HF01': ['간 건강', '간건강', '간 보호', '간손상', '간 손상', '알콜'], 'HF02': ['피로개선', '피로 개선', '에너지 이용', '에너지 생성', '탄수화물 에너지 대사', '지방 탄수화물 단백질 대사 에너지 생성'], 'HF03': ['뼈', '골다공증', '관절', '연골', '칼슘'], 'HF04': ['충치', '치아건강', '구강', '자일리톨'], 'HF05': ['면역기능', '면역 기능', '면역력', '면역조절', '세포분열', '코 상태', '코상태', '코막힘'], 'HF06': ['유해산소', '항산화', '산화', '유해산소', '유해 산소', '활성산소', '활성 산소'], 'HF07': ['피부', '상피세포'], 'HF08': ['눈 건강', '눈건강', '건조한 눈 개선', '눈 피로', '시각'], 'HF09': ['위 점막', '위 건강', '위건강', '지방 소화', '위 불편감 개선', '소화 기능', '소화기능'], 'HF10': ['배변활동', '식이섬유', '장 건강', '장내'], 'HF11': ['방광', '배뇨'], 'HF12': ['요로'], 'HF13': ['전립선'], 'HF14': ['갱년기 남성', '남성 갱년기', '정자 운동성'], 'HF15': ['갱년기 여성'], 'HF16': ['혈액 생성', '질 건강', '엽산', '철', '질 유익균 증식 유해균 억제', '월경'], 'HF17': ['결합조직', '근력', '근육', '운동수행', '운동능력', '운동수행능력', '지구성 운동', '지구력', '단백질 아미노산 이용', '에너지 포도당 지질 합성', '근육 결합조직 신체조직과도한 운동'], 'HF18': ['체지방', '체내 탄수화물 지방 단백질 대사 관여'], 'HF19': ['수면', '스트레스 긴장', '스트레스 피로'], 'HF20': ['기억력', '인지기능', '인지능력'

,HF00,HF01,HF02,HF03,HF04,HF05,HF06,HF07,HF08,HF09,...,HF17,HF18,HF19,HF20,HF21,HF22,HF23,HF24,HF25,HF26
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
3,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
432,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
433,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
434,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 영양성분 별 건강기능 코드 예외사항 - 하드코딩
- `function_dict`로 분류되지 않는 내용 관련

In [395]:
# 영양성분 별 건강기능 코드 예외사항 - 하드코딩
i_df.loc[i_df['ingredient_name'].str.contains('락추로스'), 'HF10'] = 1 # 장 건강
i_df.loc[i_df['ingredient_name'].str.contains('요오드'), 'HF16'] = 1 # 여성건강
i_df.loc[i_df['ingredient_name'].str.contains('다래추출물'), 'HF05'] = 1 # 면역개선

### 건강기능 코드 컬럼에 모든 행이 0인 경우가 있는지 확인

In [396]:
# 건강기능 코드 컬럼에 모든 행이 0인 경우가 있는지 확인하는 함수
def check_zero_value(df, code_dict):
    # 확인할 컬럼명 리스트에 담기
    check_cols = list(code_dict.keys())
    
    # 각 행별로 해당 컬럼의 모든 값이 0인지 확인하는 mask 생성
    mask = (df[check_cols] == 0).all(axis=1)
    
    # 행의 모든 값이 0인 경우 행 인덱스를 리스트에 추가
    zero_idx_list = mask[mask].index.tolist()
    
    # 분류가 하나도 되지 않은 행만 가져와 데이터프레임 생성
    check_zero_value_df = df.loc[zero_idx_list]
    
    return check_zero_value_df

In [397]:
check_zero_value(i_df, function_dict) # 건강기능 코드 컬럼이 모두 잘 분류된 것을 확인

,ingredient_raw_name,ingredient_function,ingredient_caution,ingredient_limit_high,ingredient_limit_low,ingredeint_unit,ingredient_etc,ingredient_auth_num,ingredient_name,ingredient_grp_name,...,HF17,HF18,HF19,HF20,HF21,HF22,HF23,HF24,HF25,HF26
434,기장밀추출복합물(KeranatTM)(제2024-16호),(국문) 모발상태(윤기·탄력) 개선에 도움을 줄 수 있음\n(영문) May help to improve hair condition (luster·elasticity),"(1) 영·유아, 어린이, 임산부 및 수유부는 섭취를 피할 것\n(2) 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의\n(3) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것",300,300,mg/일,제2024-16호,제2024-16호,기장밀추출복합물(KeranatTM),기장밀추출복합물,...,0,0,0,0,0,0,0,0,0,0


### 건강기능 코드 JSON 컬럼 생성
- `key` 값으로 건강기능 코드, `value` 값으로 해당 영양성분에 건강기능이 있을 경우 1, 없을 경우 0의 값을 가지는 `dict` 형태의 값을 저장하여 JSON 타입의 컬럼 생성

In [398]:
def create_json_code_col(df, code_list, code_json_col):    
    # 코드 JSON 컬럼 데이터 생성
    # key : 코드(코드컬럼명), value : 해당 영양성분 데이터프레임 행의 코드컬럼값(0 또는 1)
    code_json = []
    for idx in range(len(df)):
        code_dict_data = {}
        for code in code_list:
            code_dict_data[code] = df.at[idx, code]
        code_json.append(code_dict_data)
    
    # 코드 JSON 컬럼 생성
    df[code_json_col] = code_json
    
    return df

In [399]:
# 건강기능 코드 JSON 컬럼 생성
i_df = create_json_code_col(i_df, list(function_dict.keys()), 'ingredinet_function_code')

# 확인
i_df['ingredinet_function_code']

0      {'HF00': 0, 'HF01': 0, 'HF02': 0, 'HF03': 0, 'HF04': 0, 'HF05': 1, 'HF06': 0, 'HF07': 0, 'HF08': 0, 'HF09': 0, 'HF10': 0, 'HF11': 0, 'HF12': 0, 'HF13': 0, 'HF14': 0, 'HF15': 0, 'HF16': 0, 'HF17': 0, 'HF18': 0, 'HF19': 0, 'HF20': 0, 'HF21': 0, 'HF22': 0, 'HF23': 0, 'HF24': 0, 'HF25': 0, 'HF26': 0}
1      {'HF00': 0, 'HF01': 0, 'HF02': 1, 'HF03': 0, 'HF04': 0, 'HF05': 0, 'HF06': 0, 'HF07': 0, 'HF08': 0, 'HF09': 0, 'HF10': 0, 'HF11': 0, 'HF12': 0, 'HF13': 0, 'HF14': 0, 'HF15': 0, 'HF16': 0, 'HF17': 0, 'HF18': 0, 'HF19': 0, 'HF20': 0, 'HF21': 0, 'HF22': 0, 'HF23': 0, 'HF24': 0, 'HF25': 1, 'HF26': 0}
2      {'HF00': 0, 'HF01': 0, 'HF02': 0, 'HF03': 0, 'HF04': 0, 'HF05': 0, 'HF06': 0, 'HF07': 0, 'HF08': 0, 'HF09': 0, 'HF10': 0, 'HF11': 0, 'HF12': 0, 'HF13': 0, 'HF14': 0, 'HF15': 0, 'HF16': 0, 'HF17': 1, 'HF18': 0, 'HF19': 0, 'HF20': 0, 'HF21': 0, 'HF22': 0, 'HF23': 0, 'HF24': 1, 'HF25': 0, 'HF26': 0}
3      {'HF00': 0, 'HF01': 0, 'HF02': 1, 'HF03': 0, 'HF04': 0, 'HF05': 0, 'HF06': 0, 

## 9. 주의사항 코드 컬럼 생성
- 건강기능과 유사한 방법으로 컬럼 생성

In [400]:
from caution_dict import caution_dict
print(f'caution_dict : {caution_dict}')

i_df = create_dict_col(i_df, 'caution_nonus', caution_dict)
i_df[caution_dict.keys()]

caution_dict : {'DI01': ['위 간 신장 질환', '위 신장 간질환', '위장관 질환', '위장관질환', '위장관 장애오메프라졸', '위산분비 억제제', '위산분비억제제'], 'DI02': ['간 질환', '간질환', '간 심장질환', '간 신장 질환', '간 신장 심장질환', '간 신장 기능 이상자', '간기능 저하', '위 간 신장 질환', '위 신장 간질환', '담관'], 'DI03': ['심장', '심혈관계 질환', '심혈관약 복용', '부정맥치료제', '부정맥 치료제'], 'DI04': ['신장', '신부전'], 'DI05': ['당뇨치료제', '당뇨 천식', '당뇨병 치료 받으시는 분', '당뇨환자', '당뇨 환자', '당뇨병 있는 경우', '당뇨 혈압 질환', '항당뇨제', '혈당강하제', '당뇨 질환', '당뇨질환', '높은 혈당'], 'DI06': ['고혈압', '혈압강하제', '혈압약', '혈압 질환', '혈압질환'], 'DI07': ['고지혈증'], 'DI08': ['항응고제', '항혈소판제', '비스테로이드계 항염증약', '혈압조절제', '혈액응고', '항혈전', '혈전용해제', '출혈'], 'DI09': ['갑상선'], 'DI10': ['천식', '기관지', '해소', '기도 심한 염증'], 'DI11': ['담낭'], 'DI12': ['면역억제제', '자가면역'], 'DI13': ['수면', '신경', '항히스타민제', '항우울제', '항경련제', '항뇌전증제'], 'DI14': ['호르몬제', '에스트로겐', '대두'], 'DI15': ['단장증후군', '유당불내증'], 'DI16': ['광과민성'], 'DI17': ['호르몬제를 복용'], 'AL01': ['게', '새우'], 'AL02': ['옻'], 'AL03': ['땅콩'], 'AL04': ['프로폴리스'], 'AL05': ['대두알레르기', '대두 알레르기', '대두 난황 알레르기', '대두단백'], 'AL06': ['대두 이소플라본', '에스트로겐'], '

,DI01,DI02,DI03,DI04,DI05,DI06,DI07,DI08,DI09,DI10,...,FEMALE,MALE,S1,O3,O2,O1,단백질,비타민 A,베타카로틴,요오드
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
432,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
433,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
434,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [401]:
# 확인
i_df.loc[i_df['ingredient_caution_content'].notnull(), ['ingredient_caution_content', 'INFANT_TODDLER', 'KIDS', 'TEENAGER', 'DI05', 'AL05']]

,ingredient_caution_content,INFANT_TODDLER,KIDS,TEENAGER,DI05,AL05
1,특히 6세 이하는 과량섭취하지 않도록 주의,1,0,0,0,0
8,"의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의",0,0,0,1,0
10,대두단백에 알레르기를 나타내는 경우에는 섭취 주의,0,0,0,0,1
12,베타-카로틴(β-carotene)의 흡수를 저해할 수 있음,0,0,0,0,0
14,"어린이, 임산부, 수유기 여성은 섭취에 주의하십시오. 과량 섭취 시 복부팽만감을 느낄 수 있습니다.",1,1,0,0,0
...,...,...,...,...,...,...
431,임산부 및 수유부는 섭취를 피할 것 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의할 것 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것,0,0,0,0,0
432,"영·유아, 어린이, 임산부 및 수유부는 섭취를 피할 것 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것 질환이 있거나 의약품 복용 시 섭취 전 전문가와 상담할 것",1,1,0,0,0
433,"영·유아, 어린이, 임산부 및 수유부는 섭취를 피할 것 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의할 것 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것",1,1,0,0,0
434,"영·유아, 어린이, 임산부 및 수유부는 섭취를 피할 것 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것",1,1,0,0,0


In [402]:
# 주의사항 코드 JSON 컬럼 생성
i_df = create_json_code_col(i_df, caution_dict, 'ingredinet_caution_code')

# 확인
i_df['ingredinet_caution_code']

0      {'DI01': 0, 'DI02': 0, 'DI03': 0, 'DI04': 0, 'DI05': 0, 'DI06': 0, 'DI07': 0, 'DI08': 0, 'DI09': 0, 'DI10': 0, 'DI11': 0, 'DI12': 0, 'DI13': 0, 'DI14': 0, 'DI15': 0, 'DI16': 0, 'DI17': 0, 'AL01': 0, 'AL02': 0, 'AL03': 0, 'AL04': 0, 'AL05': 0, 'AL06': 0, 'AL07': 0, 'AL08': 0, 'AL09': 0, 'AL10': 0, 'AL11': 0, 'AL12': 0, 'AL13': 0, 'AL14': 0, 'AL15': 0, 'AL16': 0, 'AL17': 0, 'AL18': 0, 'AL19': 0, 'AL20': 0, 'P3': 0, 'P2': 0, 'P1': 0, 'INFANT_TODDLER': 0, 'KIDS': 0, 'TEENAGER': 0, 'ADULT': 0, 'OLD': 0, 'FEMALE': 0, 'MALE': 0, 'S1': 0, 'O3': 0, 'O2': 0, 'O1': 0, '단백질': 0, '비타민 A': 0, '베타카로틴': 0, '요오드': 0}
1      {'DI01': 0, 'DI02': 0, 'DI03': 0, 'DI04': 0, 'DI05': 0, 'DI06': 0, 'DI07': 0, 'DI08': 0, 'DI09': 0, 'DI10': 0, 'DI11': 0, 'DI12': 0, 'DI13': 0, 'DI14': 0, 'DI15': 0, 'DI16': 0, 'DI17': 0, 'AL01': 0, 'AL02': 0, 'AL03': 0, 'AL04': 0, 'AL05': 0, 'AL06': 0, 'AL07': 0, 'AL08': 0, 'AL09': 0, 'AL10': 0, 'AL11': 0, 'AL12': 0, 'AL13': 0, 'AL14': 0, 'AL15': 0, 'AL16': 0, 'AL17': 0, 'AL

## 10. `ingredient_id` 컬럼 생성
- PK 값인 id 값 부여

In [403]:
# i_df['ingredient_id'] = [id for id in range(1, 435)]

In [404]:
import pickle

# 이전 개발 버전의 ingredient 데이터셋의 name, id 컬럼값 사용해 merge 예정
with open('C:\\Users\\user\\working\\Our-family-pharmacist\\data\\recom_ingredient_dataset_fin_240515_v3.pkl', 'rb') as f:
    check_i_df = pickle.load(f)

In [405]:
len(i_df), len(check_i_df)

(436, 430)

- 이 전 작업에서 개별인정원료가 추가된 것 같다.

In [406]:
check_i_df.rename(columns = {'ingredient_name': 'ingredient_raw_name'}, inplace=True)
check_i_df[['ingredient_raw_name', 'ingredient_type', 'ingredient_id']]

,ingredient_raw_name,ingredient_type,ingredient_id
0,타마린드강황주정추출복합물(TamaFlexⓇ)(제2024-12호),1,1
1,저분자유청단백가수분해물(제2023-21호),1,2
2,연어이리추출물(DOTTM연어핵산)(제2019-5호),1,3
3,당귤농축분말(ROCH)(제2023-44호),1,4
4,참나리추출분말(제2023-43호),1,5
...,...,...,...
425,철,3,426
426,옥타코사놀 함유 유지,2,427
427,아연,3,428
428,요오드,3,429


In [421]:
# 확인용
check_i_df[check_i_df['ingredient_raw_name'].str.contains('풋사과')][['ingredient_raw_name', 'ingredient_id']]

,ingredient_raw_name,ingredient_id
146,풋사과추출물 애플페논(Applephenon)(제2021-6호),147
213,풋사과추출물 애플페논(Applephenon)(제2019-21호),214
341,풋사과추출물 애플페논(Applephenon)(제2015-9호),342
384,풋사과추출물 애플페논(Applephenon)(제2020-9호),385


In [408]:
new_i_df = pd.merge(i_df, check_i_df[['ingredient_raw_name', 'ingredient_id']], how='left', on='ingredient_raw_name')
# new_i_df['ingredient_id'].astype(int64)
new_i_df.loc[new_i_df['ingredient_id'].isnull(), ['ingredient_raw_name', 'ingredient_id']]

,ingredient_raw_name,ingredient_id
38,"NAG(엔에이지, N-아세틸글루코사민, N-Acetylglucosamine)",NaN
57,"엠에스엠(MSM, Methyl sulfonylmethane, 디메틸설폰)",NaN
66,"글루코만난(곤약, 곤약만난)",NaN
169,"홍삼, 사상자, 산수유 복합추출물(제2008-67호)",NaN
176,"비즈왁스알코올(Bees Wax Alcohol, BWA)(제2010-1호)",NaN
246,오메가-3 지방산 함유 유지,NaN
252,리프리놀-초록입홍합추출오일(제2009-49호),NaN
301,"피엘에이지(PLAG, 1-palmitoyl-2-linoleoyl-3-acetyl-rac-glycerol)(제2013-35호)",NaN
428,전호잎추출물(제2022-25호),NaN
429,녹용효소분해추출물(HENKIVⓇ)(제2024-2호),NaN


In [409]:
# 특수문자 등 이름이 살짝 달라 merge 되지 않은 내용 업데이트
new_i_df.loc[new_i_df['ingredient_raw_name'].str.contains('오메가'), 'ingredient_id'] = 183
new_i_df.loc[new_i_df['ingredient_raw_name'].str.contains('NAG'), 'ingredient_id'] = 391
new_i_df.loc[new_i_df['ingredient_raw_name'].str.contains('엠에스엠'), 'ingredient_id'] = 356
new_i_df.loc[new_i_df['ingredient_raw_name'].str.contains('글루코만난'), 'ingredient_id'] = 358
new_i_df.loc[new_i_df['ingredient_raw_name'].str.contains('제2008-67호'), 'ingredient_id'] = 264
new_i_df.loc[new_i_df['ingredient_raw_name'].str.contains('제2010-1호'), 'ingredient_id'] = 275
new_i_df.loc[new_i_df['ingredient_raw_name'].str.contains('제2009-49호'), 'ingredient_id'] = 174
new_i_df.loc[new_i_df['ingredient_raw_name'].str.contains('제2013-35호'), 'ingredient_id'] = 125

new_i_df.loc[new_i_df['ingredient_id'].isnull(), ['ingredient_raw_name', 'ingredient_id']]

,ingredient_raw_name,ingredient_id
428,전호잎추출물(제2022-25호),NaN
429,녹용효소분해추출물(HENKIVⓇ)(제2024-2호),NaN
430,난각막가수분해물(NEM®)(제2024-9호),NaN
431,백편두추출분말(NOVAponinⓇ)(제2024-6호),NaN
432,발효굴추출물(제2023-41호),NaN
433,풋사과 농축분말(제2024-4호),NaN
434,기장밀추출복합물(KeranatTM)(제2024-16호),NaN
435,파비플로라생강뿌리추출물(제2024-14호),NaN


In [412]:
# 댕댕이나무열매추출분말 -> 중복행 제거됨 78, 80에서 80만 사용됨 (78을 id로 가진 행 제거됨)
# 프로바이오틱스(드시모네) -> 339를 id로 가진 행 제거됨(이유 찾는중)
# 8개의 행 추가됨(2024 추가자료 등) -> 431 ~ 438을 id로 가짐
new_i_df[new_i_df['ingredient_id'].notnull()].sort_values('ingredient_id')[['ingredient_id']]

,ingredient_id
426,1.0
427,2.0
425,3.0
424,4.0
423,5.0
...,...
1,426.0
7,427.0
0,428.0
4,429.0


In [413]:
new_i_df.loc[new_i_df['ingredient_id'].isnull(), 'ingredient_id'] = [id for id in range(431, 439)]

In [414]:
# ingredient_id 열의 데이터 타입을 float64에서 int64로 변경
new_i_df['ingredient_id'] = new_i_df['ingredient_id'].astype('int64')

# 변경된 데이터 타입 확인
print(new_i_df['ingredient_id'].dtypes)
new_i_df['ingredient_id']

int64


0      428
1      426
2      424
3      425
4      429
      ... 
431    434
432    435
433    436
434    437
435    438
Name: ingredient_id, Length: 436, dtype: int64

## 11. pickle 파일 저장
- DB에 저장할 용도로 `ingredeint` 테이블과 `ingredient_com_code` 테이블에 맞는 데이터프레임 생성 후 pickle 파일로 저장

In [415]:
ingredient_table_columns = ['ingredient_id', 'ingredient_name', 'ingredient_grp_name', 'ingredient_auth_num', 'ingredient_recom_name',
                      'ingredient_type', 'ingredient_limit_high', 'ingredient_limit_low', 'ingredeint_unit',
                      'ingredient_function_content', 'ingredient_caution_content', 'ingredinet_function_code', 'ingredinet_caution_code']

ingredient_com_code_table_columns = ['ingredient_id', 'ingredinet_function_code', 'ingredinet_caution_code']


In [416]:
ingredient_table = new_i_df[ingredient_table_columns]

ingredient_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 436 entries, 0 to 435
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   ingredient_id                436 non-null    int64 
 1   ingredient_name              436 non-null    object
 2   ingredient_grp_name          436 non-null    object
 3   ingredient_auth_num          339 non-null    object
 4   ingredient_recom_name        436 non-null    object
 5   ingredient_type              436 non-null    object
 6   ingredient_limit_high        436 non-null    object
 7   ingredient_limit_low         436 non-null    object
 8   ingredeint_unit              436 non-null    object
 9   ingredient_function_content  436 non-null    object
 10  ingredient_caution_content   356 non-null    object
 11  ingredinet_function_code     436 non-null    object
 12  ingredinet_caution_code      436 non-null    object
dtypes: int64(1), object(12)
memory usag

In [417]:
from com_code import com_code_grp_dict, com_code_columns
print(com_code_grp_dict)
print(com_code_columns)

{'FUNCTION': ['HF'], 'DISEASE': ['DI'], 'ALLERGY': ['AL'], 'OPERATION': ['O'], 'PREGNANCY': ['P'], 'SMOKING': ['S'], 'AGEGROUP': ['INFANT_TODDLER', 'KIDS', 'TEENAGER', 'ADULT', 'OLD'], 'SEX': ['FEMALE', 'MALE']}
['HF00', 'HF01', 'HF02', 'HF03', 'HF04', 'HF05', 'HF06', 'HF07', 'HF08', 'HF09', 'HF10', 'HF11', 'HF12', 'HF13', 'HF14', 'HF15', 'HF16', 'HF17', 'HF18', 'HF19', 'HF20', 'HF21', 'HF22', 'HF23', 'HF24', 'HF25', 'HF26', 'DI01', 'DI02', 'DI03', 'DI04', 'DI05', 'DI06', 'DI07', 'DI08', 'DI09', 'DI10', 'DI11', 'DI12', 'DI13', 'DI14', 'DI15', 'DI16', 'DI17', 'AL01', 'AL02', 'AL03', 'AL04', 'AL05', 'AL06', 'AL07', 'AL08', 'AL09', 'AL10', 'AL11', 'AL12', 'AL13', 'AL14', 'AL15', 'AL16', 'AL17', 'AL18', 'AL19', 'AL20', 'P3', 'P2', 'P1', 'INFANT_TODDLER', 'KIDS', 'TEENAGER', 'ADULT', 'OLD', 'FEMALE', 'MALE', 'S1', 'O3', 'O2', 'O1', '단백질', '비타민 A', '베타카로틴', '요오드']


In [418]:
# ingredeint_com_code table 생성하는 함수 정의
def expand_ingredient_codes(df, pk_col, code_col, com_code_grp_dict):
    expanded_rows = []

    for idx, row in df.iterrows():
        ingredient_id = row[pk_col]
        codes_dict = row[code_col]
        
        for code, value in codes_dict.items():
            if value == 1:
                for com_code_grp, com_code_rule in com_code_grp_dict.items():
                    if re.sub(r'[0-9]', '', code) in com_code_rule:
                        expanded_rows.append({pk_col: ingredient_id, 'com_code_grp': com_code_grp, 'com_code': code})

    return pd.DataFrame(expanded_rows)

In [420]:
# 함수 적용
ingredient_com_code_function = expand_ingredient_codes(new_i_df[ingredient_com_code_table_columns], 'ingredient_id', 'ingredinet_function_code', com_code_grp_dict)
ingredient_com_code_caution = expand_ingredient_codes(new_i_df[ingredient_com_code_table_columns], 'ingredient_id', 'ingredinet_caution_code', com_code_grp_dict)

# 두 데이터프레임을 행 기준으로 합치기
ingredient_com_code_table = pd.concat([ingredient_com_code_function, ingredient_com_code_caution], axis=0)

# id 기준으로 오름차순 정렬
ingredient_com_code_table.sort_values(by='ingredient_id', ignore_index=True)
 
# 해당 테이블의 pk 값 1 ~ 1817의 값 넣기
ingredient_com_code_table['ingredient_com_code_id'] = [com_code_id for com_code_id in range(1, 1829)] # 1818 -> 1829

# 편의를 위해 컬럼 순서 재정렬
ingredient_com_code_table = ingredient_com_code_table[['ingredient_com_code_id', 'ingredient_id', 'com_code_grp', 'com_code']]

# 확인
ingredient_com_code_table

,ingredient_com_code_id,ingredient_id,com_code_grp,com_code
0,1,428,FUNCTION,HF05
1,2,426,FUNCTION,HF02
2,3,426,FUNCTION,HF25
3,4,424,FUNCTION,HF17
4,5,424,FUNCTION,HF24
...,...,...,...,...
1268,1824,438,PREGNANCY,P3
1269,1825,438,PREGNANCY,P2
1270,1826,438,AGEGROUP,INFANT_TODDLER
1271,1827,438,AGEGROUP,KIDS


In [422]:
# 피클 파일 및 csv 파일로 저장

# 피클 파일 저장
with open('C:\\Users\\user\\working\\Our-family-pharmacist\\data\\ingredient_table.pkl', 'wb') as f:
    pickle.dump(ingredient_table, f)    
    
with open('C:\\Users\\user\\working\\Our-family-pharmacist\\data\\ingredient_com_code_table.pkl', 'wb') as f:
    pickle.dump(ingredient_com_code_table, f)    
    
with open('C:\\Users\\user\\working\\Our-family-pharmacist\\data\\ingredient_processing.pkl', 'wb') as f:
    pickle.dump(i_df, f)
    
# csv 파일 저장
ingredient_table.to_csv('C:\\Users\\user\\working\\Our-family-pharmacist\\data\\ingredient_table.csv', index=False, encoding='utf-8-sig')

ingredient_com_code_table.to_csv('C:\\Users\\user\\working\\Our-family-pharmacist\\data\\ingredient_com_code_table.csv', index=False, encoding='utf-8-sig')